##### Importing Libraris

In [ ]:
# General Libraries
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 60)

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Maths and Stats
import math

# Datetime
import datetime as dt

# Exclude warnings
import warnings
warnings.filterwarnings("ignore")

# importing modeling Libraries
from sklearn.cluster import KMeans

import random
import string
import itertools

from dateutil import parser
import os
import datetime

## Decleare Functions

In [ ]:
def calculate_constraint(df = None):
    # Reformat the new clenaing timings
    df['Clean_Start_Time']= pd.to_datetime(df['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
    df['Clean_End_Time']= pd.to_datetime(df['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
    df['Usage_End']= pd.to_datetime(df['Usage_End'], format = '%d/%m/%Y %H:%M:%S')
    df['Next Usage'] =  pd.to_datetime(df['Next Usage'], format = '%d/%m/%Y %H:%M:%S')
    
    df['Actual DHT'] = df['Clean_End_Time'] - df['Usage_End']
    df['Actual DHT'] = df['Actual DHT']/(np.timedelta64(1, 's')*3600)

    # If there is NAN due to shift. fill the last clean date as the plan start date time
    df['Actual CHT'] = df['Next Usage'] - df['Clean_End_Time']
    df['Actual CHT'] = df['Actual CHT']/(np.timedelta64(1, 's')*3600)

    df['DHT Violation Flag'] = np.where((df['Actual DHT']> df['Max DHT']) | (df['Actual DHT'].isnull()), 1, 0)
    df['CHT Violation(Intermediate Gap)'] = np.where(df['Actual CHT']> df['Max CHT'], 1, 0)
    df['CHT_Adjust_Range'] = round((df['Last Cleaning_DHT'] - df['Start Cleaning_CHT'])/(np.timedelta64(1, 's')*60),0)
    cond1 = (df['Unutilized_gap'] < df['Max DHT']+df['Max CHT'])
    cond2 = (df['CHT Violation(Intermediate Gap)'] ==1)
    cond3 = (df['CHT_Adjust_Range'] > 0)
    df['CHT_Adjust'] = np.where(cond1 & cond2 & cond3, 1, 0)
    return df

In [ ]:
def CIP_assignment(df = None, index = None, main_flag = 0, val_dict = None):
    val_dict = [i for i in val_dict.items()]
    count = 0
    for val in val_dict:
        count = count+1
        if (count > 5) & (main_flag == 0):
            break
        df.loc[index, val[0]] = val[1] 
    return df

In [ ]:
def clean_allocate(df_mc = None, df_cons = None, res = '', df_update = None, index = None, loop_df= None, main_flag = 0, 
                   mc_const_df = None, clean_start = None, clean_end = None, m_start = None, m_end = None, 
                   run_flag = 0, th_time = None, reclean = 0, flag_var = '', buffer_time = None):
    
    mc_avail = check_cond(df = df_mc, col = 'MC Group', cleaning_start = clean_start, 
                          cleaning_end = clean_end)['MC Group'].unique().tolist()
    const_avail = check_cond(df = df_cons, col = 'Constraint', cleaning_start = clean_start, 
                             cleaning_end = clean_end)['Constraint'].unique().tolist()
    rows = 0
    for mc in mc_avail:
        for const in const_avail:
            
            rows = mc_const_df[(mc_const_df['Resource'] == res) & (mc_const_df['Constraint'] ==const) 
                               & (mc_const_df['MC Group'] ==mc)].shape[0]
            if rows>0:
                cl_start, cl_end = enough_gap_constraint(df = df_update.copy(), mc_group = mc,
                                                         resource = res, mins = buffer_time,
                                                         clean_start_tm = clean_start,
                                                         clean_end_tm = clean_end, th_time=th_time)
                
                cl_start, cl_end = test_overlap(df = loop_df.copy(), fin_data= final_df.iloc[:-1,:], cl_st = cl_start, 
                                                cl_end = cl_end, cip_gr = mc,const_gr= const, mins = buffer_time)
                if (clean_start > th_time):
                    rows = 0
                    continue
                if main_flag == 1:
                    print(m_start, m_end)
                dict_values = {'Clean_Start_Time': clean_start, 'Clean_End_Time': clean_end, 
                               'MC Group': mc, 'Constraint': const, flag_var: 1, 'Usage_Start': m_start, 
                               'Usage_End': m_end, 'Maint_Flag': main_flag}
                
                loop_df = CIP_assignment(df = loop_df, index = index, main_flag = main_flag, val_dict = dict_values)
                break
        if rows>0:
            break
    return  rows, loop_df

In [ ]:
def calculate_constraint_preclean(df = None):
    # temp object creationa and creating CHT timing
    temp_df = pd.DataFrame()
    for res in df.Resource.unique():
        data_temp = df[df['Resource'] == res]
        data_temp = data_temp.sort_values(['Usage_Start', 'Clean_Start_Time'])
        # Shift the data 1 level up to get perform the substraction operation
        data_temp['Last Clean End'] = data_temp['Clean_End_Time'].ffill(axis = 0).shift(1)
        temp_df = pd.concat([temp_df, data_temp])        
    df = temp_df.sort_values('Usage_End').reset_index(drop = True)
    del(temp_df)
    df_par = df[df['Resource'].isin(parellal_res)].sort_values(['Clean_Start_Time']).reset_index(drop = True)
    df = df[~df['Resource'].isin(parellal_res)]
    df_par['Last Clean End'] = df_par['Clean_End_Time'].ffill(axis = 0).shift(1)
    df_par['Last Clean End'] = np.where(df_par['Clean_Start_Time'] == df_par['Clean_Start_Time'].ffill(axis = 0).shift(1),
                                        df_par['Clean_End_Time'].ffill(axis = 0).shift(2), 
                                        df_par['Clean_End_Time'].ffill(axis = 0).shift(1))
    
    df = pd.concat([df_par, df]).sort_values('Usage_Start').reset_index(drop = True)
    
    df['Last Clean End'] = df['Last Clean End'].fillna(df.Usage_Start.min())
    df['Last CHT'] = df['Usage_Start'] - df['Last Clean End']
    df['Last CHT'] = df['Last CHT']/(np.timedelta64(1, 's')*3600)
    df['CHT Violation Flag Pre Clean'] = np.where(df['Last CHT']> df['Max CHT'], 1, 0)

    # Create the starting time slot to preclean 
    df['Pre Clean Start'] = df['Max CHT']+ df['Washing Time']
    df['Pre Clean Start'] = df['Usage_Start'] - pd.to_timedelta(df['Pre Clean Start'], unit='h')
    df['Pre Clean Start'] = pd.to_datetime(df['Pre Clean Start'], format = '%d/%m/%Y %H:%M:%S')
    return df

In [ ]:
def add_frozen_cip(pre_data, post_data):
    df = pre_data.copy()
    try:
        post_data['Clean_Start_Time'] = pd.to_datetime(post_data['Clean_Start_Time'])
        post_data['Clean_End_Time'] = pd.to_datetime(post_data['Clean_End_Time'])
        pre_data['Clean_Start_Time'] = np.nan
        pre_data['Clean_End_Time'] = np.nan
        pre_data['MC Group'] = np.nan
        pre_data['Constraint'] = np.nan
        post_data['par_clean_flag'] = np.where(post_data['Cleaning Type']=='Parallel Cleaning', 1, 0)
        for i,r in post_data.iterrows():
            res = r['Resource']
            temp_df= pre_data[pre_data['Resource'] == res].sort_values('Usage_Start')
            index_list = temp_df.index
            pre_data.loc[index_list[0],'Clean_Start_Time'] =r['Clean_Start_Time']
            pre_data.loc[index_list[0],'Clean_End_Time'] =r['Clean_End_Time']        
            pre_data.loc[index_list[0],'MC Group'] =r['MC Group']
            pre_data.loc[index_list[0],'par_clean_flag'] = r['par_clean_flag']
        pre_data['Clean_Start_Time'] = pd.to_datetime(pre_data['Clean_Start_Time'])
        pre_data['Clean_End_Time'] = pd.to_datetime(pre_data['Clean_End_Time'])
        
        pre_data['Cleaning Type'] = np.where((~pre_data['MC Group'].isnull()) & 
                                             (pre_data['Clean_Start_Time'] >= pre_data['Usage_End']),'q-Post Utilization Cleaning', 
                                              np.where((~pre_data['MC Group'].isnull()) & 
                                                     (pre_data['Clean_End_Time'] <= pre_data['Usage_Start']),'q-Pre Cleaning', np.nan))
        print(pre_data.shape)
        pre_cleaning_df = pre_data[pre_data['Cleaning Type']=='q-Pre Cleaning']
        print(pre_cleaning_df.shape)
        pre_cleaning_df[['Clean_Start_Time', 'Clean_End_Time','MC Group', 'Cleaning Type']]= np.nan
        pre_data = pd.concat([pre_data, pre_cleaning_df]).sort_values(['Resource', 'Usage_Start']).reset_index(drop =True)
        
        pre_data['Clean_Start_Time'] = np.where(pre_data['Clean_Start_Time'].isnull(),np.nan,pre_data['Clean_Start_Time'])
        pre_data['Clean_End_Time'] = np.where(pre_data['Clean_End_Time'].isnull(),np.nan,pre_data['Clean_End_Time'])
        pre_data['Clean_Start_Time'] = pd.to_datetime(pre_data['Clean_Start_Time'])
        pre_data['Clean_End_Time'] = pd.to_datetime(pre_data['Clean_End_Time'])
        return pre_data
    except:
        df['Clean_Start_Time'] = np.nan; df['Clean_End_Time'] = np.nan
        df['MC Group'] = np.nan; df['Constraint'] = np.nan; df['Cleaning Type']= np.nan
        return df

###### Variable Definition

# Initializing Variables
1. Buffer variable is provided to get a buffer time while pipes are switching from one tank to another. 
2. File names containing utlization of resources and mapping file is mentioned.
3. We have also specified the sheet names.
- raw sheet name is sheet name for utilization file we'll be creating.
4. In a list we have mentioned the resource names to be processed parallelly. Path of all input files has been stored in a variable and curent directory has been changed to the path mentioned.

In [ ]:
# Defining the variables to be used
buffer = 10
#date_before = datetime.date(2021, 8, 14)
buffer_dec = buffer/60
# File names to be read
raw_file_name = '20210713_Takeda Utilization Planned Schedule.xlsx'
map_file_name = 'Takeda Mapping Input Params.xlsx'

# Sheet to be used to read
raw_sheet_name = 'Sheet1'
map_sheet_name = 'CIP Constraint Mapping'
par_clean_sheet = 'Exception - Parelall Cleaning'

# Resource to be selected to create framework
# res_select = ['615', 'I/N 1619', 'P1', 'P2', 'P3','P4', 'SA', 'SC']

parellal_res = ['Short Line - L717S', 'AS26 TD']


# input_path= r'C:\Users\NischalKumar\OneDrive - TheMathCompany Private Limited\01. Accounts\03. Takeda\03. Data Processing\02. Planned Data\20210612-4'
input_path= r'C:\Users\AtulPoddar\OneDrive - TheMathCompany Private Limited\Documents\Takeda\testing\lot_code_test3'
# input_path= r'C:\Users\NischalKumar\OneDrive - TheMathCompany Private Limited\01. Accounts\03. Takeda\03. Data Processing\02. Planned Data\20211006'
os.chdir(input_path)

# Concatinating all input into one dataframe
1. combined all input files except CIP, ENG and down time file (yet to receive downtime file).
2. A list arr is created with all the file names except those mentioned in ignore_file list.
3. Blank dataframe master_df is created. Read each file and store it in df, creating a new column 'File Name' indicating data is picked up from which file(.TXT of file name is replaced with ' ').
4. After each loop dataframes are concatinated with the master_df. At the end index has been reset

- CIP file -- this file has data when a clean activity is already scheduled(usually empty)
- ENG batch file-- this file contains maintainance details of tanks
- Down time file-- this file would contain down time of cleaning groups-- now not in use

In [ ]:
ignore_file = ['CIP Q - CIP.TXT', 'CIP Q - ENG batch.TXT', 'CIP Q - Down Time.txt']
arr = [x for x in os.listdir() if x not in ignore_file]

master_df = pd.DataFrame()
for i in arr:
    try:
        df = pd.read_csv(i, delimiter = "\t", header = None)
    except:
        continue
    df['File Name'] = str(i).replace('.TXT', '')
    master_df = pd.concat([master_df, df])
master_df.reset_index(drop = True, inplace = True)

In [ ]:
master_df.columns

- In addT column i.e 6th column null values are imputed with 0.0. This column indicates if scheduled time needs to be prepond and  max CHT time needs to be extended.

In [ ]:
master_df[6] = pd.to_numeric(master_df[6],errors='coerce').fillna(0.0)

# Preprocessing
1. Define the column names of master_df. Set the format of usage_start and usage_end column. Many a times position of Usage_start and usage_end time are swaped, to check this we substract start time from end time, if  value is negetive store the records in dataframe master_df1. In case of postive, store it in  master_df1. 
2. swap the column values for master_df2. 
3. Divide addT by 100 to get the exact hour value. To standardize values under Type column values containing Centrifugation substing converted to Centrifugation. Get minimum time from Usage_start time and store it in today varaible. This varable is used to name the comibed data .xlsx file

In [ ]:
# master_df[5] = master_df[5].fillna(master_df[6])
#master_df[6] = pd.to_numeric(master_df[6],errors='coerce').fillna(0.0)
master_df = master_df[[0,1,2,3,4,5,'File Name',6]]
master_df.columns = ['Code', 'Resource', 'LOT', 'Usage_Start', 'Usage_End', 'Type', 'File Name','addT']

# # Converting the columns to date and time
master_df['Usage_Start']= pd.to_datetime( master_df['Usage_Start'], format = '%d/%m/%Y %H:%M:%S')
master_df['Usage_End']= pd.to_datetime(master_df['Usage_End'], format = '%d/%m/%Y %H:%M:%S')


In [ ]:
from random import randint

master_df_test_null = master_df[master_df['LOT'].isna()]
master_df = master_df[~master_df['LOT'].isna()]
res_len = len(master_df_test_null['Resource'])
lot_random_list = random.sample(range(100,1000),res_len)
lot_list = ['#DUMMY{0}'.format(i) for i in  lot_random_list]
master_df_test_null['LOT'] = lot_list 
master_df = pd.concat([master_df,master_df_test_null])

In [ ]:
utilization = (master_df['Usage_End']- master_df['Usage_Start'])/(np.timedelta64(1, 's')*3600)
master_df1 = master_df[utilization<0]
master_df2 = master_df[utilization>0]

use_end = master_df1['Usage_Start'].copy()
use_start = master_df1['Usage_End'].copy()
master_df1['Usage_Start'] = use_start
master_df1['Usage_End'] = use_end
# # Converting the columns to date and time


master_df = pd.concat([master_df1, master_df2])
master_df['Usage_Start']= pd.to_datetime( master_df['Usage_Start'], format = '%d/%m/%Y %H:%M:%S')
master_df['Usage_End']= pd.to_datetime(master_df['Usage_End'], format = '%d/%m/%Y %H:%M:%S')

master_df['addT'] = master_df['addT']/100

master_df.loc[master_df['Type'].str.contains('Centrifugation'), 'Type'] = 'Centrifugation'

today = master_df.Usage_Start.min().strftime('%Y%m%d')

# # date_after =  parser.parse('2021-08-15 00:00:00') 
# # master_df = master_df[master_df['Usage_Start']>=date_after]

# master_df.to_csv(last_modified_file_path() + '/input/' + today + '_' + utcnow() + '_Takeda_Utilization_Planned_Schedule.csv', index = False)
# master_df.shape



master_df.to_excel(today+'_Takeda Utilization Planned Schedule.xlsx', index = False)
master_df.shape

## Reading the data
- we'll get a schedule json file, if file is not present then MClist has values  ['MC1','MC2','MC3','MC4']. This file will give info. about all active MC groups.

In [ ]:
import json
try:
    f = open('schedule_input.json')
    schedule_input = json.load(f)
    f.close()
    MClist = schedule_input['data']['Active MC']
    if MClist==[]:
        MClist = ['MC1','MC2','MC3','MC4']
except Exception as e:
    print(e)
    MClist = ['MC1','MC2','MC3','MC4']

- Read the combined data .xlsx file to a pandas dataframe. Then we drop duplicates from the dataframe. Get all uniquie resource names in a list. It might contain some extra spaces hence strip the values. 

In [ ]:
# Reading the data and naming the columns
data = pd.read_excel(today+'_Takeda Utilization Planned Schedule.xlsx', sheet_name = raw_sheet_name, engine='openpyxl')
# data.columns = ['Code', 'Resource', 'LOT Code', 'Usage_Start', 'Usage_End']
data = data.drop_duplicates().reset_index(drop = True)

#Columns renaming
# data = data.rename(columns = {'Start use': 'Usage_Start', 'End use': 'Usage_End'})
# data = data.rename(columns = {'Usage Start': 'Usage_Start', 'Usage End': 'Usage_End'})

# # Converting the columns to date and time
data.Usage_Start= pd.to_datetime(data.Usage_Start, format = '%d/%m/%Y %H:%M:%S').dt.round('S').astype('datetime64[s]')
data.Usage_End= pd.to_datetime(data.Usage_End, format = '%d/%m/%Y %H:%M:%S').dt.round('S').astype('datetime64[s]')
res_select =  data.Resource.unique().tolist()
res_select = [str.strip(str(x)) for x in res_select]

# Preprocessing MC group downtime file
1. Get columns names to be assigned to schedule daaframe in a list. 
2. Open Schedule_input.json file. Read it into a dataframe and assign the column names. Column name 'Clean_Start_Time', 'Clean_End_Time' has been provided in this way so as to append it later with a dataframe such that scheduled maintainace time will be assumed as the group/pipeline is engaged in other resource cleaning.
3. ' - Group' is appended to all MC Group names

In [ ]:
down_columns = ['MC Group', 'Clean_Start_Time', 'Clean_End_Time']
try:
    f = open('schedule_input.json')
    schedule_input = json.load(f)
    f.close()
    df_downtime = pd.DataFrame(schedule_input['data']['activeMachine'])
    df_downtime = df_downtime.rename(columns = {'activeMachine': 'MC Group', 
                                                'startDate':'Clean_Start_Time', 'endDate': 'Clean_End_Time'})
    df_downtime = df_downtime[down_columns]
except Exception as e:
    df_downtime = pd.DataFrame(columns = down_columns)
    
df_downtime['Clean_Start_Time']= pd.to_datetime(df_downtime['Clean_Start_Time']).dt.tz_localize(None).astype('datetime64[s]')
df_downtime['Clean_End_Time']= pd.to_datetime(df_downtime['Clean_End_Time']).dt.tz_localize(None).astype('datetime64[s]')
df_downtime['MC Group'] = df_downtime['MC Group']+' - Group'

In [ ]:
print(df_downtime)

# Resource maintainance file
- Store column names of Maintainance dataframe in a list. Read the maintainance file to a dataframe df_maintainance, and assigne the column names. Calulate and store maintainance time span in another column.

In [ ]:
maintain_columns = ['Code', 'Resource', 'dummy', 'Start_time', 'End_time', 'Action']
try:
    df_maintenance = pd.read_csv(input_path+ '\\'+ 'CIP Q - ENG batch.TXT', delimiter = "\t", header = None)
    df_maintenance.columns = maintain_columns
    df_maintenance = df_maintenance.rename(columns = {'slno': 'Code'}).dropna(axis = 1, thresh= 1)
except:
    df_maintenance = pd.DataFrame(columns = maintain_columns)

df_maintenance.Start_time= pd.to_datetime(df_maintenance.Start_time, format = '%d/%m/%Y %H:%M:%S').astype('datetime64[s]')
df_maintenance.End_time= pd.to_datetime(df_maintenance.End_time, format = '%d/%m/%Y %H:%M:%S').astype('datetime64[s]')

df_maintenance['M_time'] = df_maintenance['End_time'] - df_maintenance['Start_time'] 
df_maintenance['M_time'] = df_maintenance['M_time']/(np.timedelta64(1, 's')*3600)

# Freezed Cleaning file
- Similar process is applied for CIP data  and resource renaming is done according business requirement

In [ ]:
cipq_columns = ['LOT', 'Resource', 'Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Cleaning Type']
try:
    df_qcip = pd.read_csv(input_path +'\\'+ 'CIP Q - CIP.TXT', delimiter = "\t", header = None)
    df_qcip = df_qcip.dropna(axis= 1, thresh= 1)
    df_qcip.columns = cipq_columns
    replace_di = {'AS26 TD Line - L2445': 'AS26 TD',  'AS26 VII Line - L2464': 'AS26 VII'}
    df_qcip.Resource = df_qcip.Resource.replace(replace_di) 
except:
    df_qcip = pd.DataFrame(columns = cipq_columns)

df_qcip['Clean_Start_Time']= pd.to_datetime(df_qcip['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S').astype('datetime64[s]')
df_qcip['Clean_End_Time']= pd.to_datetime(df_qcip['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S').astype('datetime64[s]')

- To be Continued

# Chroma/T65/Special data
1. Sort the main dataframe on basis of resource name and usage_end. 
- usage end name is given priority over usage start as we schedule cleaning process once utilization ends. Create a column named utilization whose value is usage_end- usage_start, in order to take the data in required format divide it by 3600 seconds. 
2. Now create a separate dataframe named data_chroma which is subset of main dataframe. here we consider records with resource name as chroma.
3. Now create data_T65 dataframe subset where resource name is T65
- By default resources with same LOT code, resource name and type are grouped and min usage start time,max start time and mean of addT are taken into consideration i.e these utilizations are considered as one utilization
- For resources named as T65 the condition is: we need not group the resources and on basis of LOT code and Type. Each utilization is considered different even if they have same LOT code and same Type.
4. Once the dataframes for T65 and Chroma has been created these records are removed from main dataframe
5. Then we are alining the column names for data_T65 dataframe again
6. Now we are creating a exception_resource list. These resources though have different  product Type can be considered under same utilization if their LOT id is same
7. these excetion resource records are stored in data_special dataframe
8. Type of these resources are hardcoded as 'Special Resource'
9. Now remove these resources from main data frame and store it in Data_new dataframe
10. append Data_new and data_special, reset index and store it in data dataframe
11.  Now group by the records on basis of Resource, Type, LOT. Take min of usage_start, max of usage_end, sum od utilized and mean of addT. For exception resources as the product type is same i.e Special resource hence irrespective of product type exception resources will be grouped.
12. Now append data_T65 to data, as data_T65 records need not be grouped according to business requirement.


In [ ]:
#Sorting the data inorder to get it in an expected order
data = data.sort_values(['Resource', 'Usage_End']).reset_index(drop= True)

# Calculating the duration from the available column
data['Utilized'] = data['Usage_End'] - data['Usage_Start']

# Changing the date delta form of numbers to floating number
data['Utilized'] = data['Utilized']/(np.timedelta64(1, 's')*3600)

data_chroma = data[data['Resource'] == 'Chroma']

data_T65 = data[data['Resource']=='T65']
data = data[~((data['Resource']=='T65') | (data['Resource'] == 'Chroma'))]



data_T65 =  data_T65[['Resource', 'Type', 'LOT', 'Usage_Start', 'Usage_End', 'Utilized', 'addT']]

exception_resource = ['Harvested Blow & Wash Additions', '4F Line - L2437', 'Long Transfer Line - L717L']

data_special =  data[data['Resource'].isin(exception_resource)]

data_special['Type'] = 'Special Resource'

data_new = data[~(data['Resource'].isin(exception_resource))]

data = pd.concat([data_new,data_special]).reset_index(drop =True)

data = data.groupby(['Resource','Type', 'LOT'], as_index = False).agg({'Usage_Start': 'min',
                                                                      'Usage_End': 'max',
                                                                      'Utilized': 'sum',
                                                                      'addT': 'mean'})
data = pd.concat([data, data_T65]).reset_index(drop = True)

# Mapping and chroma mapping
1. Read the mapping file(CIP Constraint Mapping sheet) and store it in mapping_df dataframe
2. Then read the revised duration file. This file contains resource name and revised duration of resources on basis of historical data which differ in value from Washing time mentioned in mapping data.
3. Then read the mapping file(Chroma sheet) and store it in map_chroma_df
4. Read the mapping file(Exception - Parelall Cleaning) and store it in map_par_clean_df
5. Rename column Product as Type in map_chroma_df
6. Now merge data_chroma(containing utilization details of all chroma resources) with map_chroma_df[Resource,Type, Process resources]. We are not considering other columns, as our mapping_df has all the information for chroma records .
- In data_chroma you'll find there is only one record for chroma resource, but when we'll join our chroma mapping with chroma utilization records we'll get 8 records against each chroma utilization record. Hence we'll perform cleaning for all product type of chroma resource.
7. Products names with Flexibles - Kit Colonna as Process resources are stored in a list
8. for these products the Type in Data_chroma data frame is defined as  Kit Colona others remain as it is
9. Now group by Data_chroma on basis of 'Process resource', 'Resource','Type', 'LOT' as we did before.
10. now there would be 4 rows(according to unique Process resource) against each chroma utilization record

# Uncomment revised duration

In [ ]:
# Pipe mapping data to be read
# mapping_path = '/dbfs/mnt/TakedaMount2/Src1/Mapping Input Params1/'
# mapping_df = pd.read_csv(mapping_path+'Takeda_Mapping_Input_Params_New_CIP_Constraint_Mapping.csv')
# map_chroma_df = pd.read_csv(mapping_path+'Takeda_Mapping_Input_Params_New_Chroma.csv')
# map_par_clean_df = pd.read_csv(mapping_path+'Takeda_Mapping_Input_Params_New_Exception_Parelall_Cleaning.csv')

mapping_path = r'C:\Users\AtulPoddar\OneDrive - TheMathCompany Private Limited\Documents\Takeda'
mapping_df = pd.read_excel(mapping_path+'\\'+map_file_name, sheet_name = map_sheet_name, engine='openpyxl')
#rev_duration = pd.read_excel(mapping_path+'\\'+'Takeda Mapping Revised Cleaning Durations.xlsx', engine='openpyxl')
#rev_duration = rev_duration[['Resource', 'Revised Cleaning Duration']]

map_chroma_df = pd.read_excel(mapping_path+'\\'+map_file_name,
                                 sheet_name = 'Chroma', engine='openpyxl')

map_par_clean_df = pd.read_excel(mapping_path+'\\'+map_file_name,
                                 sheet_name = 'Exception - Parelall Cleaning', engine='openpyxl')

map_chroma_df = map_chroma_df.rename(columns = {'Product': 'Type'})

data_chroma = pd.merge(data_chroma, map_chroma_df.iloc[:, :3], on = ['Resource', 'Type'], how = 'left')

# Product type to be added
mul_type = ["Column Product Charge","Equilibration","Eluition Chroma","Rigeneration Buffer Add", "Washing Chroma"]
data_chroma['Type'] = np.where(data_chroma['Type'].isin(mul_type), 'Kit Colona', data_chroma['Type']) 

data_chroma = data_chroma.groupby(['Process resource', 'Resource','Type', 'LOT'], 
                                  as_index = False).agg({'Usage_Start': 'min',
                                                          'Usage_End': 'max',
                                                          'Utilized': 'sum',
                                                          'addT': 'mean'})

# Active/absent MC list and Revised cleaning duartion
1. Clean the mapping df by removing unnamed columns
2. create a list allmcgroup  containing all MC groups 
3. MClist is the list we had prepared before using schedule json file, append ' - Group ' string to it
- mc_list will contain only those groups which are being used and allmcgroup list will contain all group names
4. absent_mc would contain the list of all MC groups which are not active now
5. slno column is created just for developer's reference
6. in mapping_df spaces are present in column names hence column names have been trimmed
7. Now to update the Washing time of resources with revised cleaning duration, merge mapping_df with rev_duration on resource. update washing time revised cleaning duration where revised cleaning duration is not null.
8. Drop Revised cleaning duration 

# Uncomment revised clean duration

In [ ]:
mapping_df = mapping_df[[x for x in mapping_df.columns if 'Unnamed' not in x]] # Keeping the req info
allmcgroup = ['MC1 - Group', 'MC2 - Group', 'MC3 - Group', 'MC4 - Group', 'MC5 - Group', 'MC6 - Group']
mc_list = [str(x) + ' - Group' for x in MClist]
absent_mc = [x for x in allmcgroup if x not in mc_list]
#mapping_df = mapping_df.drop(absent_mc, axis = 1)
# mapping_df = mapping_df.iloc[:46]
# Creating a slno column to follow the same order
mapping_df['slno'] = range(mapping_df.shape[0])
mapping_df['slno'] = mapping_df['slno']+1
# Removing the extra space from the column name to avoid confusions
mapping_df.columns = [x.strip() for x in mapping_df.columns]
# mapping_df = pd.merge(mapping_df, rev_duration, on='Resource', how = 'left')
# mapping_df['Washing Time'] = np.where(mapping_df['Revised Cleaning Duration'].isnull(), 
#                                       mapping_df['Washing Time'], 
#                                       mapping_df['Revised Cleaning Duration'])
# mapping_df = mapping_df.drop('Revised Cleaning Duration', axis = 1)

1. replace_di dictonary is created containing resource name which needs to be renamed as mappinf_df and utilization records differ in value. Hence replacimg the resource names in mapping_df according the dictonary
2. We had created a parallel resource list initally which contains resource names which can be considered for parallel cleaning.
3. create a a daataframe named df_special for these resouces.
4. As the pipenames assigned to these resources were merged in mapping  .xlxs file while reading this only the first record contains the value. Hence did a forward fill in df_special dataframe.
5. A separate data frame df_special was created because we cant do forward fill in mapping_df as there are many records with pipe value as NA, and that colud get affected.
6. Then remove these parallel resources from mapping_df and then merge them back

# Parallel/Special/Resource mapping preprocessing
- Context--> We usually get 1 record of flexibles resource, MC1 and MC4 group would contain V as pipe name and other groups have some other values. But there is Pipe E under MC1 and MC4 group which can clean these Flexible resource. Hence we need to insert another row for flexible resource where E is populated under MC1 and MC4 group and and other groups are NA.
- Create a blank templist list which would later contain MC1 - Group and MC2- Group.
-  Flexibles resources are stored in df_flexi_chip and a copy of this in df_flexi_chip1.
- Then we check if MC1-group is present id absent_mc list, if not then 'MC1- group' column of df_flex_chip1 is populated as E. Then add MC1- Group in templist. Similar thing is done for MC4 -Group.
- create a list named drop_mc which contais all active groups except MC1 and MC4.
- Now drop the columns present in drop_mc list from df_flex_cip1.
- concatinate df_flex_cip and df_flex_cip1
- create a dataframe temp_df from map_par_clean_df and only consider columns Resource,Type and P-clean flag
- Filter out the records containing P-clean flag as 1
- create a blank dataframe df_flexible_cip and a list named par_cl_res containing Process resource names in temp_df
- loop through each and every element in par_cl_res. Assign the process resiurce name as resource in df_flex_cip and continate it with df_flexible_cip.
- Now concatinate mapping_df with df_flexible_cip and reset the index

In [ ]:
# Resource column to make as string
mapping_df['Resource'] = mapping_df['Resource'].astype('str')

replace_di = {'AS26 TD Line - L2445': 'AS26 TD',  'AS26 VII Line - L2464': 'AS26 VII'}
mapping_df.Resource = mapping_df.Resource.replace(replace_di) 

# Treating the special resources having same constraint and can be cleaned togather
df_special = mapping_df[mapping_df.Resource.isin(parellal_res)].reset_index(drop = True)
df_special = df_special.ffill()
mapping_df = mapping_df[~mapping_df.Resource.isin(parellal_res)].reset_index(drop = True)

# Concatenating both the dataset
mapping_df = pd.concat([mapping_df,df_special])

templist=[]
df_flex_cip = mapping_df[mapping_df['Resource']=='Flexibles']
df_flex_cip1 = df_flex_cip.copy()
if 'MC1 - Group' not in absent_mc:
    df_flex_cip1['MC1 - Group'] = 'E'
    templist = ['MC1 - Group']
if 'MC4 - Group' not in absent_mc:
    df_flex_cip1[ 'MC4 - Group'] = 'E'
    templist.append('MC4 - Group')
drop_mc = [x for x in mc_list if x not in templist]
  
#df_flex_cip1[['MC1 - Group', 'MC4 - Group']] = 'E'
df_flex_cip1 = df_flex_cip1.drop(drop_mc, axis = 1)
df_flex_cip = pd.concat([df_flex_cip, df_flex_cip1])


temp_df = map_par_clean_df.iloc[:,:3]
temp_df = temp_df[temp_df['P-Clean Flag'] == 1]

df_flexible_cip = pd.DataFrame()
par_cl_res = temp_df['Process resource'].tolist()
for i in par_cl_res[2:]:
    print(i)
    df_flex_cip['Resource'] = i
    df_flexible_cip = pd.concat([df_flexible_cip, df_flex_cip])
mapping_df = pd.concat([mapping_df, df_flexible_cip]).sort_values(['slno', 'Resource']).reset_index(drop=True)

# Unpivoting parallel mapping- df_par_clean
- Melting map_par 
- PC option with value 1 are taken into consideration
- Output would be as follows:
- Resource -----  Process resource ------- PC option

In [ ]:
map_par_clean_df

In [ ]:
df_par_clean= pd.melt(map_par_clean_df, id_vars = map_par_clean_df.columns[:2],
                      value_vars=map_par_clean_df.columns[3:], var_name='PC Option')
df_par_clean = df_par_clean[df_par_clean['value']==1].iloc[:,:-1].sort_values('Process resource').reset_index(drop = True)

# Merging Raw file with Mapping file and chroma data and process resources
1. strip the resource column in utlization and mapping data.
2. store resource and process resource columns from map_par_clean_df in df_table in data dataframe
3. merge data and df_table
4. append data_chroma to data

In [ ]:
# Preparing the data for merging with mapping file (Making sure the format of mapping column same)
data['Resource'] = [str.strip(str(x)) for x in data['Resource'].tolist()]
mapping_df['Resource'] = [str.strip(str(x)) for x in mapping_df['Resource'].tolist()]

df_table = map_par_clean_df[['Resource', 'Process resource']]
data = pd.merge(df_table, data, on = 'Resource', how = 'right')

data = pd.concat([data, data_chroma], axis = 0)

In [ ]:
data_flex_4f = data[data['Resource']== '4F Line - L2437']

dup_list = ['Flexibles - Mass Capture 1/2', 'Flexibles - Mass Capture 3/4']
temp_flex = data[data['Process resource'].isin(dup_list)]

a = [x for x in dup_list if x not in temp_flex['Process resource'].unique()] 
if len(a)>0:
    data_flex_4f['Process resource'] = a[0]
else:
    for i, r in data_flex_4f.iterrows():
        temp_flex['Distance'] = (r['Usage_Start'] - temp_flex['Usage_Start'])/(np.timedelta64(1, 's')*60)

        temp_flex['time_needed1'] = temp_flex['Utilized']*60 +78+2
        temp_flex['time_needed2'] = r['Utilized']*60+78+2

        temp1 = temp_flex[(temp_flex['Distance']> 0) & (temp_flex['Distance'] < temp_flex['time_needed1'])]
        temp2 = temp_flex[(temp_flex['Distance']< 0) & (temp_flex['Distance'] > -temp_flex['time_needed2'])]

        a = [x for x in dup_list if x not in temp1['Process resource'].unique()] 
        if len(a)>0:
            data_flex_4f.loc[i, 'Process resource'] = a[0]
            continue
        a = [x for x in dup_list if x not in temp2['Process resource'].unique()]
        if len(a)>0:
            data_flex_4f.loc[i, 'Process resource'] = a[0]
            continue
            
data = pd.concat([data, data_flex_4f], axis = 0)

1. Now as mapping contains process resource names for flexible and chroma resources we need to refer process resource column in data dataframe
2. Renaming the columns, and fill na values with resource exported column
3. Merge data and Mapping_df dataframe on basis of resource. In case a resource name in mapping and data doesn't match that perticular resource will not be picked.
4. Cleaning needs to be done after a maintainance process hence when shoud have info about max DHT, to know time available after a maintainance process.
5. Sort the values in data1 on basis of priority, resource and usage_start

In [ ]:
# doubt

In [ ]:
data = data.rename(columns = {'Resource': 'Resource Exported','Process resource': 'Resource'})
data['Resource'] = data['Resource'].fillna(data['Resource Exported'])

# merging with only useful columns in the secondary table
data1 = pd.merge(data, mapping_df[['Resource', 'Priority', 'Washing Time', 'Max DHT', 'Max CHT']].drop_duplicates(), 
                 on = 'Resource', how = 'inner')
df_maintenance = df_maintenance.merge(mapping_df[['Resource','Max DHT']], on = 'Resource', how = 'left')

# Sorting the data frame based on the column names
data1 = data1.sort_values(['Priority', 'Resource', 'Usage_Start']).reset_index(drop = True)

# addT
1. Prepone usage_start time, extend max cht and utilization time on basis of addT column

In [ ]:
data1['Usage_Start'] = data1['Usage_Start'] - pd.to_timedelta(data1['addT']*3600, unit='s')
data1['Utilized'] = data1['Utilized'] + data1['addT']
data1['Max CHT'] = data1['Max CHT'] + data1['addT']

# Next utilization and unutilized
1. Get Next utilization time of a resource and calculate gap duration between two consecutive operations. This will provide information about what time slot is avaiable for us to start cleaning between two utilization.
2. Loop through each unique resource in data1 dataframe. Store records of each unique resource in data_temp, sort it on basis of usage_start.
3. create a column next usage which would contain usage start time of next usage.
4. Column Unutilized_gap is created to store the time slot available between two utilization
5. concatinate all data

In [ ]:
data1 = data1.sort_values(['Resource', 'Usage_Start', 'LOT', 'Usage_End'])

# empty data frame to be used to concatenate in every iteration
temp_df = pd.DataFrame()
#looping through each resource to calculate the gap duration between two consicutive operation
for res in data1.Resource.unique():
    data_temp = data1[data1['Resource'] == res]
    data_temp = data_temp.sort_values('Usage_Start')
    # Shift the data 1 level up to get perform the substraction operation
    data_temp['Next Usage'] = data_temp['Usage_Start'].shift(-1)
    data_temp['Unutilized_gap'] = data_temp['Next Usage'] - data_temp['Usage_End']
    temp_df = pd.concat([temp_df, data_temp])

data2 = temp_df.reset_index(drop = True)
del(temp_df)

# Changing the date delta form of numbers to floating number
data2['Unutilized_gap'] = data2['Unutilized_gap']/(np.timedelta64(1, 's')*3600)

##### Formation of Raw dataset from the mapping file
- Creating a dataframe to be used in optimization, i.e.
- Get resource name, and all active group columns from mapping_df and store it in df_cip_gconst dataframe
- Now melt df_cip_gconst, we'll get Resource, MC group, values
- Remove records where value is null
- Create a MC matrix which contains MC group, no of unique resources under each group and no. of unique pipes assiciated with it
- const_matrix contains no. of resources associated with each pipe.
- drop duplicates from df_cip_gconst_temp ane rename the column value to constraint

In [ ]:
select_var=['Resource']+allmcgroup
df_cip_gconst = mapping_df.loc[:, select_var]
df_cip_gconst = pd.melt(df_cip_gconst, id_vars = df_cip_gconst.columns[0], value_vars=df_cip_gconst.columns[1:], var_name='MC Group')


In [ ]:
mapping_df

In [ ]:
# Creating a temporary data frame to hold the melted data

# Creating a slno column to follow the same order
df_cip_gconst = pd.merge(df_cip_gconst, mapping_df[['Resource', 'slno']], how = 'left', on = 'Resource')
df_cip_gconst = df_cip_gconst[~df_cip_gconst['value'].isnull()]
df_cip_gconst_all = df_cip_gconst.rename(columns = {'value': 'Constraint'}).drop_duplicates()
df_cip_gconst= df_cip_gconst[~df_cip_gconst["MC Group"].isin(absent_mc)].reset_index(drop=True)
df_cip_gconst_temp = df_cip_gconst.drop_duplicates()
df_cip_gconst_temp = df_cip_gconst_temp.rename(columns = {'value': 'Constraint'})

# Creation of matrix needed for MC group matrix
mc_matrix = df_cip_gconst.groupby('MC Group', as_index = False).agg({'Resource': pd.Series.nunique,
                                                            'value': pd.Series.nunique})
# Creation of matrix needed for Group constraint
const_matrix = df_cip_gconst.groupby('value', as_index = False).agg({'Resource': pd.Series.nunique})

### Data Freezing

Context

- There might be few cases when our schedules and cleaning may have been carried forward from previous week or continued after last week schedule time frame.
- These cleanings are included in our CIP data for this week and are identified as freezed cleaning. These cleaning cant be changed.
- Our code will tag these utilization with the schedules available provided for this week.
- In case we find any information about previous utilization then its a post cleaning otherwise its tagged as precleaning
- There will be few cases where two cleanings will be scheduled in between two utilization. In this case the 1st cleaning is tagged as post cleaning and 2nd cleaning as pre cleaning

1. Create a copy of data2 in util_data
2. Create a column DHT_Time - This column would give us the information about by when cleaning should complete after a utilization. This will help us identifying a freezed post cleaning for a utilization if any.
2. Impute null values of Next usage with DHT time
3. dropping DHT_Time??

In [ ]:
util_data= data2.copy()
util_data['DHT_Time'] = util_data['Usage_End'] + pd.to_timedelta(data2['Max DHT']*3600, unit='s')
util_data['Next Usage'] = util_data['Next Usage'].fillna(util_data['DHT_Time'])
util_data = util_data.drop('DHT_Time', axis = 1)
util_data['Cleaning Type'] = np.nan
qpre_data = df_qcip.copy()

1. check if qpre_data contains any record,
  - yes ---> then next step 
  - else ----> create columns 'Clean_Start_Time','Clean_End_Time', 'MC Group','Constraint','Cleaning Type'
2. preclean_rep dataframe is created which will be used later to store freezed cleaning records not tagged as post cleaning
3. Loop through each and every record of CIP and get the index and row values of each record
4. res variable stores resource name of each record
5. clean_start stores Clean_Start_Time of each record
6. temp_df dataframe stores subset of util_data where resource name = res and sort by Utlization start
7. temp_df_1 is subset of temp_df where clean_start_time > Usage _end time and next usage start time > usage start time and cleaning type is null
8. store index of the above record in index_list list
9. if index list not 0 then create and update columns 'Clean_Start_Time','Clean_End_Time','MC Group' and 'Cleaning Type'= 'q-Post Utilization Cleaning' in util_data
10. if index list is o the concatinate the recorde with preclean_rep
11. once the first loop is done for all cip records, iter through preclean_rep records.
12. Same as above except another record would go under condition temp_df['Usage_Start'] > clean_end
13. In case there are two cleanings in between utilization there would two records with 1st utilization one with post cleaning record and one with pre cleaning record 

In [ ]:
## test

In [ ]:
qpre_data.to_clipboard()

In [ ]:
qpre_data

In [ ]:
#### test

In [ ]:
if qpre_data.shape[0]>0:
    preclean_rep= pd.DataFrame()
    for i,r in qpre_data.iterrows():
        res = r['Resource']
        clean_start = r['Clean_Start_Time']

        temp_df= util_data[util_data['Resource'] == res].sort_values('Usage_Start')
        temp_df_1 = temp_df[(temp_df['Usage_End'] <= clean_start) & (temp_df['Next Usage'] > clean_start)
                            & (temp_df['Cleaning Type'].isnull())]

        index_list = temp_df_1.index.to_list()
        if len(index_list)==0:
            preclean_rep = pd.concat([preclean_rep, qpre_data.loc[[i],:]])
            print("Data unavailable for post cleaning")
        else:
            util_data.loc[index_list[0],'Clean_Start_Time'] =r['Clean_Start_Time']
            util_data.loc[index_list[0],'Clean_End_Time'] =r['Clean_End_Time']
            util_data.loc[index_list[0],'MC Group'] =r['MC Group']
            util_data.loc[index_list[0],'Cleaning Type'] ='q-Post Utilization Cleaning'

    for i,r in preclean_rep.iterrows():
        res = r['Resource']
        clean_end = r['Clean_End_Time']; 
        clean_start = r['Clean_Start_Time']

        temp_df = util_data[util_data['Resource'] == res].sort_values('Usage_Start')
        temp_df_1 = temp_df[(temp_df['Usage_Start'] > clean_end)].sort_values('Usage_Start')

        index_list = temp_df_1.index.to_list()
        if len(index_list)==0:
            print('Data Unavailable in the List ('+ str(i) + ',' + str(res) +")")
        else:
            rec_rec = util_data.loc[index_list[0]]
            rec_rec['Clean_Start_Time'] = clean_start
            rec_rec['Clean_End_Time'] = clean_end
            rec_rec['MC Group'] = r['MC Group']
            rec_rec['Cleaning Type'] = 'q-Pre Cleaning'
            util_data = pd.concat([util_data, rec_rec.to_frame().T])
    util_data = util_data.sort_values(['Resource', 'Usage_Start', 'Clean_Start_Time']).reset_index(drop = True)
    convert_columns = ['Utilized', 'addT', 'Priority', 'Washing Time', 'Max DHT', 'Max CHT', 'Unutilized_gap']
    util_data[convert_columns] = util_data[convert_columns].apply(pd.to_numeric)
    
else:
    util_data['Clean_Start_Time'] = np.nan
    util_data['Clean_End_Time'] = np.nan
    util_data['MC Group'] = np.nan
    util_data['Constraint'] = np.nan
    util_data['Cleaning Type'] = np.nan

In [ ]:
util_data.to_clipboard()

1. Copy updated Util data 
2. Create Map_table dataframe which is a subset of df_cip_gconst_temp(unpivoted datagframe of mapping dataframe) containing columns Resource, MC-Group and Constraint
3. Removing records from map_table where constraint is E and resource starts with Flex so thatfreezed flexible resources are not allocated both v and e constraints
4. rename column Constraint to Const
5. Remove ' - Group' from MC Group name

In [ ]:
df = util_data.copy() # add_frozen_cip(pre_data= data2.copy(), post_data = df_qcip.copy()).replace('nan', np.nan)
map_table = df_cip_gconst_all[['Resource', 'MC Group', 'Constraint']]
map_table = map_table[(map_table['Constraint']!='E') | (~map_table['Resource'].str.contains('Flex'))]
map_table = map_table.rename(columns = {'Constraint': 'Const'})
map_table['MC Group'] = map_table['MC Group'].str.replace(' - Group', '')


# Pre allocated Flexibles
1. Segrigating allocated cleaning records and unallocated one into df_pre and df_post dataframes
2. Merge Df_pre with map_table to get Constraint details
3. df_flex_special datframe is created containing records where MC_group is 1 and 4 and resources should be a Flexible resource.
4. Consider only 'Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint' columns
- in df_flex_special we have freezed cleanings of flexible resources handeled by MC1 and MC2 group
- This is done to handel cleaning allocation for fleible resources as these resources have 2 set of constraints under MC1 and MC2

In [ ]:
df_pre = df[~df['MC Group'].isnull()]
df_post = df[df['MC Group'].isnull()]
df_pre = pd.merge(df_pre, map_table, on = ['Resource', 'MC Group'], how = 'left')
df_pre['Constraint'] = df_pre['Const']
df_pre = df_pre.drop('Const', axis = 1)

df_flex_special = df_pre[(df_pre['MC Group'].isin(['MC1', 'MC4'])) & 
                         (df_pre.Resource.str.contains('Flex'))].sort_values('Clean_Start_Time')
df_flex_special = df_flex_special[['Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint']].drop_duplicates()


1. create dataframe df_flex_par_temp by grouping Clean_start_time, clean_end_time and MC_group in df_pre dataframe.
2. Filter records where count is >1. This will provide us details about resources with parallel freezed cleanings.
3. Create a column named Parallel cleaning flag which is like assiging sereal no to the records in df_flex_par_temp. 
4. storing the max no of parallel cleaning in last_par_count variable

In [ ]:
df_flex_par_temp = df_pre.groupby(df_flex_special.columns[:-1].tolist(), as_index = False)['Resource'].count()
df_flex_par_temp = df_flex_par_temp[df_flex_par_temp['Resource']==2]
df_flex_par_temp['Parallel Clean Flag'] = list(range(1, df_flex_par_temp.shape[0]+1))
try:
    last_par_count = int(df_flex_par_temp['Parallel Clean Flag'].max())
except:
    last_par_count = 0

# Pre allocated flexibles overlap
1. if df_flex_special contains any record next steps are followed
- These process is done to check if tehre is an overlap between two flexi cleaning.
- by default V is assigned to all flexi resource, but in case of any overlap the 2nd cleaning will be assigned to E.
- in case there are three records with an overlap, i.e 1st two records havea overlap and 2nd 3rd resource have an overlap then allocation of only 2nd resource needs to be changed. 3rd one will already be allocated to V.
- Note: all flexible resources have washing duration of 78 minutes.
2. Create a column 'last_clean_gap' which is diffrence between next resource cleaning start time and present record cleaning start time.
- as we know time duration of cleaning for a flexi resource is 79 mins 1st record is assigned 79 mins.
- in case of overlap the diffrence between two consicutive start time will be <79 mins
3. create a column last_const where last cleaning constraint is taken into consideration.
4. Assign the constrain E in case of overlap.

In [ ]:
if df_flex_special.shape[0]>0:
    df_flex_special['last_clean_gap'] = df_flex_special['Clean_Start_Time']-df_flex_special['Clean_Start_Time'].shift(1)
    df_flex_special['last_clean_gap'] = df_flex_special['last_clean_gap']/(np.timedelta64(1, 's')*60)
    df_flex_special['last_clean_gap'] = df_flex_special['last_clean_gap'].fillna(79)
    df_flex_special['last_const'] = df_flex_special['Constraint'].shift(1)
    count = 0
    for i, r in df_flex_special.iterrows():
        df_flex_special['last_const'] = df_flex_special['Constraint'].shift(1)
        if count == 0:
            count = count+1
            continue
        last_const = df_flex_special['last_const'][i]
        print(last_const)
        if (r['last_clean_gap']<78) & (last_const == r['Constraint']):
            const =[x for x in ['V', 'E'] if x not in [last_const]][0]
            print(const)
            df_flex_special.loc[i,'Constraint'] = const
            df_pre.loc[(df_pre['MC Group'] == r['MC Group']) & 
                       (df_pre['Clean_Start_Time'] == r['Clean_Start_Time']), 'Constraint'] = const

# Parallel Clean flag
1. Concat df_pre, df_post to create a final dataframe after frew cleanings 
2. Handel null values
3. append ' - Group ' string to ['MC Group'] column
4. take a copy of df in data2 dataframe
5. append ' - Group ' string to ['MC Group'] column in df_flex_par_temp
6. merging dara2 with df_flex_par_temp on 'Clean_Start_Time', 'Clean_End_Time', 'MC Group' this will give us the parallel cleaning flag in data2 dataframe

In [ ]:
df = pd.concat([df_pre, df_post], axis = 0).reset_index(drop = True)
df = df.replace('nan', np.nan).replace('NaN', np.nan)
df['MC Group'] = df['MC Group']+' - Group'
data2 = df.copy()
data2['Clean_Start_Time'] = pd.to_datetime(data2['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
data2['Clean_End_Time'] = pd.to_datetime(data2['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')

df_flex_par_temp['MC Group'] = df_flex_par_temp['MC Group'] + ' - Group'

data2 = pd.merge(data2, df_flex_par_temp[['Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Parallel Clean Flag']], 
                 on = ['Clean_Start_Time', 'Clean_End_Time', 'MC Group'], how = 'left')

In [ ]:
#data2.to_clipboard()

##### Working with the mapping file

In [ ]:
# data2['Max DHT'] = data2['Max DHT']+(10/60)

# creating few necessary columns
1. Few necesary columns are required :
2. Clean_start column- usage_end + 1 min
- Clean start time is from what time the clean process can be scheduled
3. clean end time- for the time it is null
4. Ideal time = unutilized gap - washing time
- Ideal time indicates duration when resource is nethier utilized nor washed
5. Run flag = 1 and clean flag = 1
- For the time all the records will hold run flag and clean flag as 1
- in case of recleaning(later part of code) flag will be 0
6. Max DTH = Utlization end + max DTH
7. Last possible cleaning = next usage - Washing time
- This indicates max possible start time to clean a resource
- For few records this value can be null in case its the last utilization of a resource as Next usage will be null
8. Last cleaning DHT - 'Max DHT Time' - 'washing time'
- on basis of DHT what is the max possible start time to clean a resource
9. Cond= last possible clean> last cleaning dht or last possible cleaning is null
- we cant take last cleaning DHT time in case where maxt DHT exeeds next utilization start time.
- In above case next utilization start time needs to be considered.
- hence created above condition to check the same
10. LAST Possible clean = if last possible clean> last clean dth or last possible clean is null then assign last cleaning DTH else last possible clean.
11. Start Cleaning CHT - Next Usage - washing time
 - on basis of CHT what is earliest possible start time of cleaning


In [ ]:
# Creation of cleaning start just after the first buffer is finished
data2['clean_start'] = data2['Usage_End'] + pd.Timedelta(minutes = 1)
data2['clean_end'] = np.nan

# Calculation of available hours post removing the buffer time
data2['idle_time'] = data2['Unutilized_gap']-data2['Washing Time']; data2['run_flag'] = 1; data2['clean_flag'] = 1

data2['Max DHT Time'] = data2['Usage_End'] + pd.to_timedelta(data2['Max DHT']*3600, unit='s')
data2['Last Possible Clean'] = data2['Next Usage'] - pd.to_timedelta(data2['Washing Time']*3600, unit='s')
data2['Last Cleaning_DHT'] = data2['Max DHT Time'] - pd.to_timedelta(data2['Washing Time']*3600, unit='s')

data2[data2['Resource']=='P2'].tail()

cond = (data2['Last Possible Clean']> data2['Last Cleaning_DHT']) | (data2['Last Possible Clean'].isnull())

data2['Last Possible Clean'] = np.where(cond, data2['Last Cleaning_DHT'], data2['Last Possible Clean'])

data2['Start Cleaning_CHT'] = data2['Next Usage'] - pd.to_timedelta(data2['Max CHT']*3600, unit='s')
data2['Start Cleaning_CHT'] = data2['Start Cleaning_CHT'] - pd.to_timedelta(data2['Washing Time']*3600, unit='s')

# Can create a column to see possible amount of time where it can run this

# Handeling high priority resources
1. Res_adjust- list of some important resources
- These resources have long cleaning time, hence hardly violates max CHT. And maximum time we'll find their CHT and DHT are overlapped
- For these resources we need to avoid recleaning, as there is a high chance we'll not get a recleaning slot due to above reason
2. Cond1 = start cleaning CHT > clean start and resource is in res_adjust
   Cond2 = start cleaning CHT is not null and 
   - Start cleaning CHT should be greater than clean start as there might be few cases where CHT time will exceed time gap between two utilization
   - start cleaning_CHT should not be null -- this is required as there won't be any next utilization time, if we dont keet this condition recleanings might get alloted.
   - resources should belong to res_adjust list
   - start cleaning CHT < last possible clean
3. Only if the above condition satisfies assign clean_start = Start clean_cht otherwise clean CHT
4. Clean end time = clean_start + washing time
5. last possible pre cleaning = ---- doubt

In [ ]:
res_adjust = ['P1', 'P2', 'P3', 'P4', 'SA', 'SB', 'SC', 'SD', 'AS26 TD',
              'I/N 97', 'I/N 2224', 'I/N 2667','I/N 2315', 'I/N 1619','I/N 2668','I/N 2316','I/N 1554', 
              'Dome 121', 'Dome 89', 'Dome 2215', '617', '618', '619']

cond1 = (data2['Start Cleaning_CHT']> data2['clean_start']) & (data2['Resource'].isin(res_adjust)) 
cond2 = (~data2['Start Cleaning_CHT'].isnull()) & (data2['Start Cleaning_CHT']< data2['Last Possible Clean'])
# data2['Cond'] = cond1
data2['clean_start'] = np.where(cond1 & cond2, data2['Start Cleaning_CHT'], data2['clean_start'])
# Assuming the cleaning time is exactly as the defined time
data2['clean_end'] = data2['clean_start'] + pd.to_timedelta(data2['Washing Time']*3600, unit='s')

data2['Last_pos_preclean'] = data2['Usage_Start'] - pd.to_timedelta(data2['Washing Time']*3600, unit='s') - pd.Timedelta(minutes = buffer/5)
data2['Day Flag'] = np.where(data2['clean_start'].dt.date!=data2['Last Cleaning_DHT'].dt.date, 1, 0)

Creating Matrix to set priority
- perc_rem_time --- smaller value means high priority resource as its less flexible.
- resource with less ideal time has more priority
- resource with less avaible time has more importance
- resource with more washing time has more imp.
- resource with more dht has more imp.

In [ ]:
# Data filtered only slected resources
data_selected = data2[data2['Resource'].isin(res_select)]
data_selected1 = data_selected[['Resource', 'Day Flag', 'clean_start', 'clean_flag']]

# Creation of idle time and utilization matrix at resource level
data_matrix = data_selected.groupby('Resource', as_index= False).agg({'Unutilized_gap': 'mean', 
                                                                      'run_flag': 'sum',
                                                                      'Washing Time': 'mean',
                                                                      'Max CHT': 'mean',
                                                                      'Max DHT': 'mean'})


# Creation of %idel time metric based on the washing time and the idle time between 2 cycles
data_matrix['perc_rem_time'] = 1 - (data_matrix['Washing Time']/data_matrix['Unutilized_gap'])
data_matrix['perc_rem_time'] = data_matrix['perc_rem_time'].round(2)

# Creation of same day flag for generating the flexibility factor at daily level
data_selected1.clean_start = data_selected1.clean_start.dt.date
data_selected_pivot = pd.pivot_table(data = data_selected1.copy(), values = 'clean_flag', 
                                     index = ['clean_start', 'Day Flag'], columns = 'Resource',
                                     aggfunc = 'sum', dropna = False)

data_clean_freq_matrix = data_selected_pivot.fillna(0).sum(axis = 1).reset_index(name ='Total Freq')

data_clean_freq_matrix['day_index'] = data_clean_freq_matrix['clean_start'] - data_clean_freq_matrix['clean_start'].min()
data_clean_freq_matrix['day_index'] = (data_clean_freq_matrix['day_index']/(np.timedelta64(1, 's')*3600))/24

# Creation of column to have values such as 'day index, the schedule index'
data_clean_freq_matrix['day_flag_index'] = data_clean_freq_matrix['day_index'] + data_clean_freq_matrix['Day Flag']
data_clean_freq_matrix.day_index = data_clean_freq_matrix.day_index.astype('int').astype('str')
data_clean_freq_matrix.day_flag_index  = data_clean_freq_matrix.day_flag_index.astype('int').astype('str')
data_clean_freq_matrix['day_index'] = data_clean_freq_matrix['day_index']+','+data_clean_freq_matrix['day_flag_index']


# Craetion of flexibility using all possible metrics
data_matrix['idle_flex_fact'] = data_matrix.Unutilized_gap.rank(method = 'dense', ascending = True)
data_matrix['util_flex_fact'] = data_matrix.run_flag.rank(method = 'dense', ascending = False)
data_matrix['aval_flex_fact'] = data_matrix['perc_rem_time'].rank(method = 'dense', ascending = True)
data_matrix['wash_flex_fact'] = data_matrix['Washing Time'].rank(method = 'dense', ascending = False)
data_matrix['DHT_flex_fact'] = data_matrix['Max DHT'].rank(method = 'dense', ascending = True)
 

data_clean_freq_matrix['flex_factor'] = data_clean_freq_matrix['Total Freq'].rank(method = 'dense', ascending = False)
data_clean_freq_matrix.drop(['day_flag_index', 'Day Flag'], axis = 1, inplace = True)

# Craetion of flexibility factor at CIP group level
mc_matrix['res_p_const'] =  mc_matrix['Resource']/mc_matrix['value']
mc_matrix['flex_fact'] = mc_matrix.res_p_const.rank(method = 'dense', ascending = False)

# Craetion of flexibility factor at Constraint level
const_matrix['flex_fact'] = const_matrix.Resource.rank(method = 'dense', ascending = True)
const_matrix.rename(columns = {'value': 'Constraint', 'Resource': '#Resource'}, inplace = True)


###### Final Flexibility factor created
- In order to consider every flexibility factor and to take a decision, a final flexiblity factor is very important
- Final flexibility factor is going to be the avergae of all the flexibility factor calculated
- Dorting the data metrics using the final flexibility factor
- The order post sorting says the priority of the resources

In [ ]:
# Creation of final data metrics
data_matrix['final_flex_fact'] = (data_matrix['idle_flex_fact']+data_matrix['util_flex_fact']+
                                 data_matrix['aval_flex_fact']+data_matrix['wash_flex_fact']+
                                 data_matrix['DHT_flex_fact'])/5
data_matrix['final_flex_fact'] = data_matrix['final_flex_fact'].rank(method = 'dense', ascending = True)
data_matrix = data_matrix.sort_values('final_flex_fact')

# Resource ranking
1. df_resource1 contains unique no. of MC groups and unique no. of pipes
2. total working hours = mx usage time of  datafram - min usage time of dataframe
3. df_resource2 -> group by resource and aggregate 'Utilized': ['sum', 'mean'],                                              'Unutilized_gap': 'sum','Washing Time': ['sum', 'mean'],'run_flag': 'sum','Max CHT': 'mean','Max DHT': 'mean'
4. '%Utilized'= 'Utilized sum']/total_working_hrs
5. '%idle_postclean' = 1- (df_resource2['Washing Time sum']/df_resource2['Unutilized_gap sum'])
6. merge resource1 and respource 2

In [ ]:
# Group by to have a dataset with information related to CIP at resource level
df_resource1 = df_cip_gconst.groupby('Resource', as_index =False).agg({'MC Group': pd.Series.nunique,
                                                             'value': pd.Series.nunique})
total_working_hrs = data2.Usage_End.max() - data2.Usage_Start.min()
total_working_hrs = total_working_hrs/(np.timedelta64(1, 's')*3600)

# Group by to have a dataset with information related to resource and associated metrics
df_resource2 = data2.groupby('Resource', as_index = False).agg({'Utilized': ['sum', 'mean'], 
                                                                'Unutilized_gap': 'sum',
                                                                'Washing Time': ['sum', 'mean'],
                                                                'run_flag': 'sum', 
                                                                'Max CHT': 'mean', 
                                                                'Max DHT': 'mean'})
# Handling column mmulti index
df_resource2.columns = [' '.join(x).strip() for x in df_resource2.columns]

# creation of additional columns which will help segment the groups very well
df_resource2['%Utilized'] = df_resource2['Utilized sum']/total_working_hrs
df_resource2['%idle_postclean'] = 1- (df_resource2['Washing Time sum']/df_resource2['Unutilized_gap sum'])
df_resource2.drop(['Utilized sum', 'Washing Time sum', 'Unutilized_gap sum'], axis = 1, inplace= True)

# Merging of the columns to get one final dataset
df_resource = pd.merge(df_resource1, df_resource2, how = 'right', on  = 'Resource')

1. Replace all infinite values with 1 under %idle_postclean
2. assign priority- if %idle_postclean < 0.6 then 1,elseif <0.85 then 2 else 3 1 highest and 3 being least priority
3. assign the resources in parellal_res as priority 1
4. flexible are least important hence 1 ia added to max priority value

In [ ]:
df_resource = df_resource.sort_values('%idle_postclean')
df_resource['%idle_postclean'] = df_resource['%idle_postclean'].replace([np.inf, -np.inf], 1)
df_resource['Priority'] = df_resource['%idle_postclean'].apply(lambda x: 1 if x < 0.6 else (2 if x < 0.85 else 3))
df_resource['Priority'] = np.where(df_resource.Resource.isin(parellal_res), 1, df_resource['Priority'])
df_resource.loc[df_resource['Resource'].str.contains('Flexible'), 'Priority'] = df_resource['Priority'].max()+1

1. Sort values on basis of matrix
2. give reource rank on basis of 'Priority', '%idle_postclean', 'Max DHT mean', 'Max CHT mean', 'value', 'Washing Time mean'

In [ ]:
df_resource = df_resource.sort_values(['Priority', '%idle_postclean', 'Max DHT mean', 'Max CHT mean', 'value', 'Washing Time mean'], 
                                        ascending = [True, True, True, True, True, False])

df_resource['Resource Rank'] = range(1,df_resource.shape[0]+1)
res_order = df_resource.Resource.tolist()

1. Drop priority
2. merge the priorities and ranking with main dataframe

In [ ]:
data2 = data2.drop('Priority', axis = 1)
data2 = pd.merge(data2, df_resource[['Resource', 'Priority', 'Resource Rank']], on = 'Resource', how = 'inner')
df_cip_gconst = pd.merge(df_cip_gconst, df_resource[['Resource', 'Priority']], on = 'Resource', how = 'inner')

cip_list = ['MC1 - Group', 'MC2 - Group']
df_cip_gconst['Group'] = np.where(df_cip_gconst['MC Group'].isin(cip_list), 1, 2)

df_cip_gconst_bkp = df_cip_gconst.copy()

data_bkp = data2.copy()

In [ ]:
def cip_avail(df = None, sort_by_col = '', for_col = '', sub_col = 'Clean_End_Time', max_clean = None):
    map_columns = ['MC Group', 'Constraint', 'Clean_Start_Time', 'Clean_End_Time']
#     df_cip_new = df[map_columns].drop_duplicates().copy()
    df_par = df[~df['Parallel Clean Flag'].isnull()]
    df_non_par = df[df['Parallel Clean Flag'].isnull()]
    df_par = df_par[map_columns].drop_duplicates().copy()
    df_non_par = df_non_par[map_columns]
    df_cip_new = pd.concat([df_non_par, df_par], axis = 0).reset_index(drop = True)
    if for_col=='MC Group':
        df_cip_new = pd.concat([df_downtime, df_cip_new])
    
    df_cip_new['Clean_Start_Time'] = pd.to_datetime(df_cip_new['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
    df_cip_new['Clean_End_Time'] = pd.to_datetime(df_cip_new['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
    
    temp_df = pd.DataFrame()
    for val in df_cip_new[for_col].unique():
        data_temp = df_cip_new[df_cip_new[for_col] == val]
        data_temp = data_temp.sort_values(sort_by_col)
        data_temp['idle_till_time'] = data_temp[sort_by_col].shift(-1)
        data_temp['idle_till_time'] = data_temp['idle_till_time'].fillna(max_clean)
        temp_df = pd.concat([temp_df, data_temp])
    
    temp_df['idle_till_time'] = pd.to_datetime(temp_df['idle_till_time'], format = '%d/%m/%Y %H:%M:%S')
    temp_df[sub_col] = pd.to_datetime(temp_df[sub_col], format = '%d/%m/%Y %H:%M:%S')
    temp_df = temp_df.sort_values([for_col, sort_by_col]).reset_index(drop = True)
    temp_df['Available_Time']  = ((temp_df['idle_till_time'] - temp_df[sub_col])/(np.timedelta64(1, 's')*3600))*60
    return temp_df

In [ ]:
# Function to get the added column with information related to availability for recleaning and last cleaning time
def cleaning_availablity(df = None, sort_by_col = '', for_col = '', sub_col = 'Clean_End_Time', max_clean = None, 
                         clean_dur = 0, df_avail = None, buffer_mins = 0):
    map_columns = [for_col, 'Clean_Start_Time', 'Clean_End_Time']
    df_par = df[~df['Parallel Clean Flag'].isnull()]
    df_non_par = df[df['Parallel Clean Flag'].isnull()]
    df_par = df_par[map_columns].drop_duplicates().copy()
    df_non_par = df_non_par[map_columns]
    df = pd.concat([df_par, df_non_par])
    df_cip = df[map_columns]
    df_cip = df[map_columns]
    temp_df = df.groupby(for_col, as_index = False).agg({'Clean_Start_Time':'min'})
    temp_df['Clean_End_Time'] = min_clean
    
    if for_col=='MC Group':
        df_cip_new = pd.concat([temp_df,df_downtime, df_cip, df_avail])
    else:
        df_cip_new = pd.concat([temp_df, df_cip, df_avail])
    
#     df_cip_new = df_cip_new.drop_duplicates()
    df_cip_new = df_cip_new[~df_cip_new[for_col].isnull()]
    df_cip_new['Clean_Start_Time'] = pd.to_datetime(df_cip_new['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
    df_cip_new['Clean_End_Time'] = pd.to_datetime(df_cip_new['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
    temp_df = pd.DataFrame()
    
    for val in df_cip_new[for_col].unique():
        data_temp = df_cip_new[df_cip_new[for_col] == val]
        data_temp = data_temp.sort_values(sort_by_col)
        data_temp['idle_till_time'] = data_temp[sort_by_col].shift(-1)
        data_temp['idle_till_time'] = data_temp['idle_till_time'].fillna(max_clean)
        temp_df = pd.concat([temp_df, data_temp])
    temp_df['idle_till_time'] = pd.to_datetime(temp_df['idle_till_time'], format = '%d/%m/%Y %H:%M:%S')
    temp_df[sub_col] = pd.to_datetime(temp_df[sub_col], format = '%d/%m/%Y %H:%M:%S')
    temp_df = temp_df.sort_values([for_col, sort_by_col]).reset_index(drop = True)
    temp_df['Available_Time']  = (temp_df['idle_till_time'] - temp_df[sub_col])/(np.timedelta64(1, 's')*3600)
    temp_df['Last_pos_Clean'] = temp_df['idle_till_time'] - pd.Timedelta(minutes=clean_dur*60+buffer_mins)
    return temp_df

In [ ]:
def check_cond(df = None, col = None, cleaning_start = None, cleaning_end = None):
    wash_time = (cleaning_end - cleaning_start)/(np.timedelta64(1, 's')*3600)
    cond1 = (df['Last_pos_Clean'] > cleaning_start)
    cond2 = (df['Available_Time'] > wash_time)
    cond3 = (df['Clean_End_Time'] < cleaning_start)
    final_cond = cond1 & cond2 & cond3
    list_out = df[final_cond].sort_values('Available_Time', ascending = False)
    return list_out

In [ ]:
def test_overlap(df = None, fin_data= None, cl_st = None, cl_end = None, cip_gr = None, const_gr= None, mins = None):
    dict_values = {'Clean_Start_Time': [cl_st], 'Clean_End_Time': [cl_end], 'MC Group': [cip_gr], 
                    'Constraint': [const_gr], 'Initial Assignment': [1], 'Usage_Start': [None], 
                    'Usage_End': [None], 'Maint_Flag': [0]}
    df = CIP_assignment(df = df.copy(), index = index, main_flag = 0, val_dict = dict_values)
    temp_d = df.loc[[index],:].copy()
    fin_data = pd.concat([fin_data, temp_d])
    df_cip_o= cleaning_availablity(df = fin_data.copy(), sort_by_col = 'Clean_Start_Time', 
                                            sub_col =  'Clean_End_Time', for_col = 'MC Group', max_clean = max_clean, 
                                            clean_dur = washing_time, df_avail = df_cip_avail.copy(), buffer_mins = mins)

    df_const_o = cleaning_availablity(df = fin_data.copy(), sort_by_col = 'Clean_Start_Time', 
                                            sub_col =  'Clean_End_Time', for_col = 'Constraint', max_clean = max_clean, 
                                            clean_dur = washing_time, df_avail = df_const_avail.copy(), buffer_mins = 0)
    
    c_overlap = df_const_o[df_const_o['Available_Time']<0].shape[0]
    d_violation = fin_data[fin_data['Last Possible Clean']< fin_data['Clean_Start_Time']].shape[0]
    m_overlap = df_cip_o[df_cip_o['Available_Time']<0].shape[0]

    if (c_overlap + d_violation + m_overlap) > 0:
        cl_st = th_time+pd.Timedelta(minutes = 1)
    return cl_st, cl_end

In [ ]:
def enough_gap_constraint(df = None, mc_group = None, resource = None, mins = 0, 
                          clean_start_tm = None, clean_end_tm = None, th_time = None):
    max_date_df = df[(df['MC Group'] == mc_group) & (df['Resource'] != resource)]
    if max_date_df.shape[0]>0:
        max_date_df = max_date_df[max_date_df['Clean_End_Time'] <= clean_start_tm]
    max_date = max_date_df['Clean_End_Time'].max()
    washing_time = (clean_end_tm - clean_start_tm)/(np.timedelta64(1, 's')*60)
    if max_date_df.shape[0] > 0:
        time_gap = (clean_start_tm - max_date)/(np.timedelta64(1, 's')*60)
    else:
        time_gap = mins
    if time_gap >= mins:
        clean_start_tm = clean_start_tm
        clean_end_tm = clean_end_tm
    else:
        clean_start_tm_temp = max_date + pd.Timedelta(minutes = mins)
        if clean_start_tm_temp >= th_time:
            clean_start_tm = clean_start_tm
        else:
            clean_start_tm = clean_start_tm_temp
        clean_end_tm = clean_start_tm + pd.Timedelta(minutes = washing_time)
    return (clean_start_tm, clean_end_tm)

In [ ]:
def Push_Cleaning_Transfermation1(df = None, clean_end = None, clean_start = None):

    df['Clean_Start_Time'] = pd.to_datetime(df['Clean_Start_Time'].fillna(pd.NaT), errors='coerce', 
                                            format = '%d/%m/%Y %H:%M:%S')
    df['Clean_End_Time'] = pd.to_datetime(df['Clean_End_Time'].fillna(pd.NaT), errors='coerce',
                                          format = '%d/%m/%Y %H:%M:%S')
    
    df['Last Possible Clean'] = pd.to_datetime(df['Last Possible Clean'].fillna(pd.NaT), errors='coerce', 
                                               format = '%d/%m/%Y %H:%M:%S')
    drop_col = ['idle_till_time', 'Last_pos_Clean']

    cip_df = cleaning_availablity(df = df.copy(), sort_by_col = 'Clean_Start_Time', sub_col =  'Clean_End_Time', 
                                  for_col = 'MC Group', max_clean = max_clean, clean_dur = washing_time,
                                  df_avail = df_cip_avail.copy(), buffer_mins = buffer/5).drop(drop_col, axis = 1)

    const_df = cleaning_availablity(df = df.copy(), sort_by_col = 'Clean_Start_Time', sub_col =  'Clean_End_Time', 
                                    for_col = 'Constraint', max_clean = max_clean, clean_dur = washing_time,
                                    df_avail = df_const_avail.copy(), buffer_mins = 0).drop(drop_col, axis = 1)


    cip_df =cip_df.rename(columns = {'Available_Time': 'MC_Available_Time'})
    const_df = const_df.rename(columns = {'Available_Time': 'C_Available_Time'})
    
    df_merged = pd.merge(df, cip_df, on = ['MC Group','Clean_Start_Time', 'Clean_End_Time'], how = 'left')
    df_merged = pd.merge(df_merged, const_df, on = ['Constraint','Clean_Start_Time', 'Clean_End_Time'], how = 'left')
    
    df_merged[['MC_Available_Time','C_Available_Time']] = df_merged[['MC_Available_Time','C_Available_Time']]*60 - (washing_time*60)
    
    df_merged[['MC_Available_Time','C_Available_Time']] = df_merged[['MC_Available_Time','C_Available_Time']].round(2)
    
    df_merged['Clean_gap'] = (clean_end - df_merged['Clean_Start_Time'])/(np.timedelta64(1, 's')*60)
    df_merged['Clean_st_gap'] = (df_merged['Clean_End_Time'] - clean_start)/(np.timedelta64(1, 's')*60)
    
    df_merged['idle post buffer'] = df_merged['Last Possible Clean'] - df_merged['Clean_Start_Time']
    df_merged['idle post buffer'] = df_merged['idle post buffer']/(np.timedelta64(1, 's')*60)
    return df_merged

In [ ]:
def Push_Cleaning_Transfermation(df = None, clean_end = None, clean_start = None):

    df['Clean_Start_Time'] = pd.to_datetime(df['Clean_Start_Time'].fillna(pd.NaT), errors='coerce', 
                                            format = '%d/%m/%Y %H:%M:%S')
    df['Clean_End_Time'] = pd.to_datetime(df['Clean_End_Time'].fillna(pd.NaT), errors='coerce',
                                          format = '%d/%m/%Y %H:%M:%S')
    df['Last Possible Clean'] = pd.to_datetime(df['Last Possible Clean'].fillna(pd.NaT), errors='coerce', 
                                               format = '%d/%m/%Y %H:%M:%S')

    cip_df = cleaning_availablity(df = df.copy(), sort_by_col = 'Clean_Start_Time', sub_col =  'Clean_End_Time', 
                                  for_col = 'MC Group', max_clean = max_clean, clean_dur = washing_time,
                                  df_avail = df_cip_avail.copy(), buffer_mins = buffer/5)

    const_df = cleaning_availablity(df = df.copy(), sort_by_col = 'Clean_Start_Time', sub_col =  'Clean_End_Time', 
                                    for_col = 'Constraint', max_clean = max_clean, clean_dur = washing_time,
                                    df_avail = df_const_avail.copy(), buffer_mins = 0)


    cip_df =cip_df.rename(columns = {'idle_till_time':'MC_idle_till_time', 'Available_Time': 'MC_Available_Time',
                                                         'Last_pos_Clean': 'MC_Last_pos_Clean'})
    df_merged = pd.merge(df, cip_df, on = ['MC Group','Clean_Start_Time', 'Clean_End_Time'], how = 'left')
    df_merged['MC_Available_Time'] = df_merged['MC_Available_Time'] - washing_time
    df_merged['MC_Available_Time'] = df_merged['MC_Available_Time']*60
    df_merged['MC_Available_Time'] = df_merged['MC_Available_Time'].round(2)
    df_merged['Clean_gap'] = (clean_end - df_merged['Clean_Start_Time'])/(np.timedelta64(1, 's')*60)
    df_merged['Clean_st_gap'] = (df_merged['Clean_End_Time'] - clean_start)/(np.timedelta64(1, 's')*60)
    df_merged['idle post buffer'] = df_merged['Last Possible Clean'] - df_merged['Clean_Start_Time']
    df_merged['idle post buffer'] = df_merged['idle post buffer']/(np.timedelta64(1, 's')*60)
    return df_merged

In [ ]:
def Push_Cleaning_condition(df = None, usage_end = None):
    cond1 = (df['Clean_gap']>0) 
#     | ((df['Clean_gap']>=0) & (df['idle_time']>=1)) 
    cond2 = (df['idle post buffer'] > df['Clean_gap'])
    cond3 = (df['MC_Available_Time'] > 0)
    cond4 = (df['Pre_Exist'] != 1)
    cond = cond1 & cond2 & cond3 & cond4
    df_push = df[cond].sort_values('Clean_gap').reset_index(drop = True)
    return df_push

In [ ]:
def Push_Cleaning_condition_st(df = None, usage_end = None):
    df['clean_dist1'] = (clean_start - df['Clean_Start_Time'])/(np.timedelta64(1, 's')*60)
    df['clean_dist'] = df['clean_dist1']+ washing_time*60
    cond1 = (df['Clean_st_gap']>0) 
    cond2 = (df['idle post buffer'] > df['clean_dist'])
    cond3 = (df['clean_dist1']>=0)
    cond4 = (df['Pre_Exist'] != 1)
    cond = cond1 & cond2 & cond3 & cond4
    df = df.drop('clean_dist1', axis = 1)
    df_push = df[cond].sort_values('clean_dist').reset_index(drop = True)
    return df_push

In [ ]:
def Push_Cleaning_CIP(df = None, for_col = None, element  = '', start_time = None):
    
    df_push=  df[(df[for_col]==element) & (df['Clean_Start_Time'] >= start_time)].sort_values('Clean_Start_Time')
    a = df_push.groupby([for_col, 'Clean_Start_Time', 'Clean_End_Time'], as_index = False)['MC_Available_Time'].min()
    b = df_push.groupby([for_col, 'Clean_Start_Time', 'Clean_End_Time'], as_index = False)['idle post buffer'].min()
    df_push = df_push.drop(['MC_Available_Time', 'idle post buffer'], axis = 1)
    df_push = pd.merge(df_push, a, how = 'left', on = [for_col, 'Clean_Start_Time', 'Clean_End_Time'])
    df_push = pd.merge(df_push, b, how = 'left', on = [for_col, 'Clean_Start_Time', 'Clean_End_Time'])
    df_push['available_time'] = np.where(df_push['idle post buffer']< df_push['MC_Available_Time'],
                                         df_push['idle post buffer'], df_push['MC_Available_Time'])-buffer/5
    
    df_push['available_time_cs'] = df_push['available_time'].cumsum()
    
    return df_push

In [ ]:
def Push_Cleaning_Setuptime(last_mc = None, last_const = None, start_time = None, clean_start = None, clean_end =None):
    buffer_val = buffer/5
    max_time = max(last_mc, last_const)
    av_time = (clean_start - max_time)/(np.timedelta64(1, 's')*60)
    added_time = 0
    if av_time < buffer_val:
        clean_start = max_time + pd.Timedelta(minutes = buffer_val)
        clean_end = clean_start + pd.Timedelta(minutes = washing_time*60)
        added_time = buffer_val - av_time
    return added_time, clean_start, clean_end

In [ ]:
def Push_Cleaning_Updated(df = None, add_min = 0, clean_min = 0):
    df_backup = df.copy()
    df = df.loc[:df[df['available_time'] >= add_min].index[0]]
    df['Clean_Start_Time_New'] = df['Clean_Start_Time']+ pd.Timedelta(minutes = add_min)
    df['Clean_End_Time_New'] = df['Clean_End_Time'] + pd.Timedelta(minutes = add_min)
    list_par = df['Parallel Clean Flag'].dropna().unique().tolist()
    if sum(list_par)>0:
        df = pd.concat([df, df_backup[df_backup['Parallel Clean Flag'].isin(list_par)]]).sort_values('Clean_Start_Time')
        df[['Clean_Start_Time_New', 'Clean_End_Time_New']] = df[['Clean_Start_Time_New', 'Clean_End_Time_New']].ffill()
        df = df.drop_duplicates()
        
    return df

In [ ]:
def Push_Cleaning_final_ds(main_df = None, cip_df = None):
    main_df = pd.merge(main_df, cip_df[['Resource', 'LOT', 'Usage_Start', 'Clean_Start_Time_New', 'Clean_End_Time_New', 'clean_start']], 
                on = ['Resource', 'LOT', 'Usage_Start', 'clean_start'], how = 'left')
    main_df['Clean_Start_Time'] = np.where(main_df['Clean_Start_Time_New'].isnull(), main_df['Clean_Start_Time'], 
                                           main_df['Clean_Start_Time_New'])
    main_df['Clean_End_Time'] = np.where(main_df['Clean_End_Time_New'].isnull(), main_df['Clean_End_Time'], 
                                           main_df['Clean_End_Time_New'])

    col_list = main_df.columns.to_list()
    if 'MC_Available_Time' in col_list:
        col_index = col_list.index('MC_Available_Time') + 1

    main_df = main_df.iloc[:,:col_index]
    
    return main_df

In [ ]:
def Push_Cleaning(df = None, clean_start = None, clean_end = None, washing_time = None, usage_end = None, 
                  usage_start = None, index_val = 0, last_pos = None):
    df = df.sort_values(['Usage_End', 'Usage_Start', 'Clean_Start_Time']).reset_index(drop = True).copy()
    df = parallel_suffle(df =  df.copy(), clean_start_time = clean_start)
    df1 = Push_Cleaning_Transfermation1(df = df.copy(), clean_end = clean_end, clean_start = clean_start)
    last_index = df1[(df1['clean_start']==clean_start) & (df1['clean_end']==clean_end) & (df1['Resource']==res)].index[-1]
#     last_index = df1.index[-1]
    df_clean_push = Push_Cleaning_condition(df = df1.copy(), usage_end = usage_end)
    df_backup = df1.copy(); cl_start = clean_start; cl_end = clean_end
    max_clean_pos = max_clean; min_clean_pos = min_clean; flag = 1
    
    for i in range(df_clean_push.shape[0]):
        df1 = df_backup.copy()
        try:
            df_clean_pushi = df_clean_push.iloc[i]  
            push_clean_start = df_clean_pushi['Clean_Start_Time'];   clean_gap = df_clean_pushi['Clean_gap']
            push_mc =  df_clean_pushi['MC Group'];      #push_const = df_clean_pushi['Constraint']
            push_mc_list = df_cip_gconst[df_cip_gconst['Resource'] ==res]['MC Group'].tolist()
            if push_mc not in push_mc_list:
                continue
            push_const_list = df_cip_gconst[(df_cip_gconst['Resource'] ==res) & 
                                            (df_cip_gconst['MC Group'] ==push_mc)]['Constraint'].tolist()
            df_const_push_temp = df1.groupby('Constraint', as_index = False)['Clean_End_Time'].max()
            df_const_push_temp = df_const_push_temp[df_const_push_temp['Constraint'].isin(push_const_list)]
            if df_const_push_temp.shape[0] == 0:
                push_const = push_const_list[0]
                min_end = min_clean_pos
                df1['C_Available_Time'] = (max_clean_pos - min_end)/(np.timedelta64(1, 's')*60) - (washing_time*60)
            else:
                min_end = df_const_push_temp['Clean_End_Time'].min()
                push_const = df_const_push_temp[df_const_push_temp['Clean_End_Time']==min_end]['Constraint'].tolist()[0]

#             df1['C_Available_Time'] = (max_clean_pos - min_end)/(np.timedelta64(1, 's')*60) - (washing_time*60)
            df1['MC_Available_Time'] = np.where(df1['C_Available_Time']<df1['MC_Available_Time'], 
                                               df1['C_Available_Time'], df1['MC_Available_Time'])


            last_used_mc = df1[(df1['MC Group']==push_mc) & (df1['Clean_End_Time'] <= push_clean_start)]['Clean_End_Time'].max()
            last_used_const = df1[(df1['Constraint']==push_const) & (df1['Clean_End_Time'] <= push_clean_start)]['Clean_End_Time'].max()

            df_push_mc = Push_Cleaning_CIP(df = df1.copy(), for_col = 'MC Group', 
                                           element  = push_mc, start_time = push_clean_start)
            df_push_const = Push_Cleaning_CIP(df = df1.copy(), for_col = 'Constraint', 
                                               element  = push_const, start_time = push_clean_start)
            if df_push_const.empty:
                df_push_const = df_push_mc.copy()
            
            added_time = Push_Cleaning_Setuptime(last_mc = last_used_mc, last_const = last_used_const, start_time = push_clean_start,
                                        clean_start = cl_start, clean_end = cl_end)
            adjust_needed = clean_gap + added_time[0] + (buffer/5)
            clean_start = added_time[1]
            clean_end = added_time[2]
            if clean_start > last_pos:
                continue
#             df_push_mc = Push_Cleaning_Updated(df = df_push_mc.copy(), add_min = adjust_needed, clean_min = washing_time)
#             df_push_const = Push_Cleaning_Updated(df = df_push_const.copy(), add_min = adjust_needed, clean_min = washing_time)
            df_push_mc = consecutive_push(df_push = df_push_mc, df_main = df1.copy(), focus = 'MC Group', gap = adjust_needed,
                                          push_mc = push_mc, push_const = push_const)
            df_push_const = consecutive_push(df_push = df_push_const,  df_main = df1.copy(), focus = 'Constraint',
                                             gap = adjust_needed, push_mc = push_mc, push_const = push_const)

            df = Push_Cleaning_final_ds(main_df = df1, cip_df = df_push_mc)
            df = Push_Cleaning_final_ds(main_df = df, cip_df = df_push_const)
            
            df = df.drop('MC_Available_Time', axis = 1)

            df.loc[last_index, 'Clean_Start_Time'] = added_time[1]
            df['Clean_Start_Time'] = pd.to_datetime(df['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')

            df.loc[last_index, 'Clean_End_Time'] = added_time[2]
            df['Clean_End_Time'] = pd.to_datetime(df['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')

            df.loc[last_index, 'MC Group'] = push_mc
            df.loc[last_index, 'Constraint'] = push_const
            df_cipc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
                          sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

            df_constc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
                          sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

            const_overlap = df_constc[df_constc['Available_Time']<0].shape[0]
            dht_violation = df[df['Last Possible Clean']< df['Clean_Start_Time']].shape[0]
            mc_overlap = df_cipc[df_cipc['Available_Time']<0].shape[0]
        
            if (const_overlap + dht_violation + mc_overlap) > 0:
                continue
            flag = 0
            break
        except:
            flag = 1
            continue
    if flag ==1:
        raise ValueError('A very specific bad thing happened.')
    return df

In [ ]:
def Utilization_Shifter(df = None):   
    cond1 = (df['CHT Violation Flag Pre Clean']== 1)
    cond2 = (df['Last CHT'] > df['Max CHT'])

    cond3 = (df['Resource'].isin(['615', '616', '617', 'I/N 1619']))   

    cond4 = (df['Type'] == '2nd Susp. Buffer')
    cond5 = (df['Resource'].isin(['I/N 2667', 'I/N 2668']))
    diff = df['Last CHT']- (24 -(df['Usage_End'] - df['Usage_Start'])/(np.timedelta64(1, 's')*3600))
    cond6 = (diff < df['Max CHT'])
    
    df['Utilization Shifted'] = 0
    df_filt  = df[(cond1 & cond2 & cond6) & (cond3 | (cond4 & cond5))]
    df = df[~((cond1 & cond2 & cond6) & (cond3 | (cond4 & cond5)))]
    
    df_filt['Utilization Shifted'] = 1

    df_filt['Usage_Start'] = df_filt['Usage_Start'] - pd.to_timedelta((df_filt['Last CHT'] - df_filt['Max CHT'])*3600, unit='s')
    df_filt['Usage_Start'] = df_filt['Usage_Start'] - pd.Timedelta(minutes = 10)
    df = pd.concat([df, df_filt]).sort_values(['Usage_End', 'Clean_Start_Time']).reset_index(drop = True)
    return df

In [ ]:
def parallel_suffle(df =  None, clean_start_time = None):
    df_par_iden = df[(~df['Parallel Clean Flag'].isnull())].sort_values('Parallel Clean Flag', ascending = False)
    for par in df_par_iden['Parallel Clean Flag'].unique():
        df_par_ds = df[df['Parallel Clean Flag']==par].sort_values('Last Possible Clean')
        index_val = df_par_ds.index
        df_sel_pc = df_par_ds.loc[index_val[0]]
#         df.loc[index_val[1], 'Parallel Clean Flag'] = np.nan

        pc_option = df_par_clean[df_par_clean['Process resource']==df_sel_pc['Resource']]['PC Option'].tolist()

        df_pos_par = df[(df["Resource"].str.startswith('Flexible')) & (df['Parallel Clean Flag'].isnull()) &
                         (df['Clean_Start_Time']> df_sel_pc['Usage_End']) & (df['Resource'].isin(pc_option)) & 
                         (df['Clean_Start_Time']<= df_sel_pc['Last Possible Clean'])]
        
        if df_pos_par.shape[0]>=1:
            df.loc[index_val[1], 'Parallel Clean Flag'] = np.nan

            new_index_val = df_pos_par.sort_values('Clean_Start_Time').index    
            df.loc[new_index_val[0], 'Parallel Clean Flag'] = par
            df_pos_par.loc[new_index_val[0], 'Parallel Clean Flag'] = par
            df_pos_par = df_pos_par.iloc[0]

            col_list = ['Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint', 'Parallel Clean Flag']
            for col in col_list:
                df.loc[index_val[0], col] = df_pos_par[col]
    return df

In [ ]:
def consecutive_push(df_push = None, df_main = None, focus = 'MC Group', gap = 0, push_mc = None, push_const = None):
    c_avail = df_push['C_Available_Time'][0]
    mc_avail = df_push['MC_Available_Time'][0]
    df_push['Clean_Start_Time_New'] = np.nan; df_push['Clean_End_Time_New'] = np.nan; df_temp = pd.DataFrame()
    if (c_avail>=gap) & (mc_avail>=gap):
        print('Life Sorted')
        df_push.loc[0,'Clean_Start_Time_New'] = df_push['Clean_Start_Time'][0]  + pd.Timedelta(minutes = gap)
        df_push.loc[0,'Clean_End_Time_New'] = df_push['Clean_End_Time'][0]  + pd.Timedelta(minutes = gap)

    else:
        for push_index in range(df_push.shape[0]):

            c_avail = df_push['C_Available_Time'][push_index]
            mc_avail = df_push['MC_Available_Time'][push_index]

            if focus == 'MC Group':
                element_focus = mc_avail; element_alter = c_avail; push_el = df_push['Constraint'][push_index]
                el_foc = push_const
            else:
                element_focus = c_avail; element_alter = mc_avail; push_el = df_push['MC Group'][push_index]
                el_foc = push_mc
            print(push_el)
            df_push.loc[push_index,'Clean_Start_Time_New'] = df_push['Clean_Start_Time'][push_index]  + pd.Timedelta(minutes = gap)
            df_push.loc[push_index,'Clean_End_Time_New'] = df_push['Clean_End_Time'][push_index]  + pd.Timedelta(minutes = gap)

            if (element_focus >= gap) &  (element_alter >= gap):
                df_push = df_push.iloc[:push_index+1, :]
                break
            elif (element_alter < gap) & (push_el != el_foc):
                df_temp = df1[(df1['Constraint'] == df_push['Constraint'][push_index]) & 
                              (df1['Clean_Start_Time'] > df_push['Clean_Start_Time'][push_index])].sort_values('Clean_Start_Time')
                df_temp['Clean_Start_Time_New'] = df_temp['Clean_Start_Time'] + pd.Timedelta(minutes = gap)
                df_temp['Clean_End_Time_New'] = df_temp['Clean_End_Time'] + pd.Timedelta(minutes = gap)
            df_push = pd.concat([df_push, df_temp])
    df_push['Clean_Start_Time_New'] = pd.to_datetime(df_push['Clean_Start_Time_New'], format = '%d/%m/%Y %H:%M:%S')
    df_push['Clean_End_Time_New'] = pd.to_datetime(df_push['Clean_End_Time_New'], format = '%d/%m/%Y %H:%M:%S')
    return df_push

# Recleaning
1. Sort the datframe on basis of resource, usage_start, cleaing type
- sorting in decending manner with Cleaning type will help to get  the freezed records first and unallocated one at the end.
2. Create a blank dataframe df_new
3. Loop all unique resources
4. df_temp = subset of dataframe where resource name = res
5. Prev_last_pos = last_possible_cleaning of previous utilization and fill 1st record with minimum clean time.
6. Last_pos_clean = Clean start time of previous record(except freezed data this value will be null)
7. LastPos_clean = if Last_pos_clean is null then Prev_Last_Pos else LastPos_clean.
- in case we have a clean start time of previous utilization take that else pick the last possible cleaning 
- in case we have a freezed data we'll get a clean start time
8. Time Gap = Usage_Start - Last_Pos_Clean
- time gap will provide us the time gap between the utilization start and previous utilization's cleaning start time
- we'll use this info. later to compare with CHT time, which will give us an idea if we'll violate max CHT
9. Deduct washing time from time gap as
10. Concatinate with df_new
11. come out of loop
12. Pre clean_start = usage_start - max_CHT - washing time
13.  Pre clean_start if Pre_clean_start > last possible clean the pre clean start else last pos clean
- this is to make sure after a reclean max CHT time is not violated
14. create rec_iden_df--> recleaning identifying df if Time Gap > max CHT and Cleaning Type not equal to pre cleaning
15. Take records where clean start time is null. This means we are removing all freez data
16. Assign clean type as recleaning
17. Last possible clean - usage start - washing time
18. clean start = pre clean start
19. clean end = clean start = washing
20. concatinate this data frame with df_new
- This means addtional rowas are added for recleanings


In [ ]:
def include_recleans(df= None):
    df = df.sort_values(['Resource','Usage_Start', 'Cleaning Type'],  ascending=[True,True, False]).reset_index(drop = True)
    df_new = pd.DataFrame()
    for res in df['Resource'].unique():
        df_temp = df[df['Resource'] == res]
        df_temp['Prev_Last_Pos'] = df_temp['Last Possible Clean'].shift(1).fillna(min_clean)
        df_temp['Last_Pos_Clean'] = df_temp['Clean_Start_Time'].shift(1)
        df_temp['Last_Pos_Clean'] = np.where(df_temp['Last_Pos_Clean'].isnull(), df_temp['Prev_Last_Pos'],
                                            df_temp['Last_Pos_Clean'])
        df_temp['Time Gap'] = df_temp['Usage_Start'] - df_temp['Last_Pos_Clean']
        df_temp['Time Gap'] = df_temp['Time Gap']/(np.timedelta64(1, 's')*3600)
#         df_temp = df_temp[df_temp['Cleaning Type'] != 'q-Pre Cleaning']
        df_temp['Time Gap'] = df_temp['Time Gap'] - df_temp['Washing Time']
        df_new = pd.concat([df_new, df_temp]).reset_index(drop = True)
        
    
    df_new['Pre Clean Start'] = df_new['Max CHT']+ df_new['Washing Time']
    df_new['Pre Clean Start'] = df_new['Usage_Start'] - pd.to_timedelta(df_new['Pre Clean Start'], unit='h')
    df_new['Pre Clean Start'] = pd.to_datetime(df_new['Pre Clean Start'], format = '%d/%m/%Y %H:%M:%S') 
    df_new['Pre Clean Start'] = np.where(df_new['Pre Clean Start'] > df_new['Last_Pos_Clean'], 
                                         df_new['Pre Clean Start'], df_new['Last_Pos_Clean'])
    
    rc_iden_df = df_new[(df_new['Time Gap'] > df_new['Max CHT']) & (df_new['Cleaning Type'] != 'q-Pre Cleaning')]
    rc_iden_df = rc_iden_df[rc_iden_df['Clean_Start_Time'].isnull()]
    rc_iden_df['Cleaning Type'] = 'Reclean'
    rc_iden_df['Last Possible Clean'] = rc_iden_df['Usage_Start'] - pd.to_timedelta(rc_iden_df['Washing Time']*3600, unit='s')
    rc_iden_df['clean_start'] = rc_iden_df['Pre Clean Start']
    rc_iden_df['clean_end'] = rc_iden_df['clean_start']+pd.to_timedelta(rc_iden_df['Washing Time']*3600, unit='s')
    df_new = pd.concat([df_new, rc_iden_df]).reset_index(drop = True)
    return df_new.copy()

In [ ]:
def check_overlaps(df = None):

    df_cipc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
                  sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

    df_constc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
                  sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

    const_overlap = df_constc[df_constc['Available_Time']<0].shape[0]
    dht_violation = df[df['Last Possible Clean']< df['Clean_Start_Time']].shape[0]
    mc_overlap = df_cipc[df_cipc['Available_Time']<0].shape[0]

    return const_overlap, dht_violation, mc_overlap

In [ ]:
def cip_positions(df = None, var = None):
    df_mis = df[df[var].isnull()]
    df = df[~df[var].isnull()]
    print(df_mis.shape)
    temp_df = pd.DataFrame()
    for mc in df[var].unique():
        df_mc = df[df[var]==mc].sort_values('Clean_End_Time')
        df_mc['Last_'+var+'_End'] = df_mc['Clean_End_Time'].shift(1).fillna(min_clean)
        df_mc['Next_'+var+'_Start'] = df_mc['Clean_Start_Time'].shift(-1).fillna(max_clean)
        temp_df = pd.concat([temp_df, df_mc], axis = 0)
    df = pd.concat([temp_df, df_mis])
    return df

In [ ]:
def process_missing_parallel(df = None, count = None):
    df_flex = df[(df['Resource'].str.contains('Flex')) & (df['Parallel Clean Flag'].isnull())]
    df_flex_miss = df_flex[df_flex['MC Group'].isnull()].sort_values('Last Possible Clean')
    for flex_ind, flex_row in df_flex_miss.iterrows():
        clean_start = flex_row['clean_start']
        clean_end = flex_row['clean_end']
        last_poss = flex_row['Last Possible Clean']
        flex_res = flex_row['Resource']
        
        pc_option = df_par_clean[df_par_clean['Process resource']==flex_res]['PC Option'].tolist()
        
        df_pos = df_flex[(~df_flex['MC Group'].isnull()) & 
                         (df_flex['Clean_Start_Time']< last_poss) &
                         (df_flex['Clean_Start_Time']> clean_start) &
                         (df_flex['Pre_Exist']!=1) &
                         (df_flex['Resource'].isin(pc_option)) &
                         (df_flex['Parallel Clean Flag'].isnull())].sort_values('Last Possible Clean')
#         print(flex_res, df_pos.shape)
        if df_pos.shape[0]>0:
            count = count + 1
            print('Resource:', flex_res, 'Count:', df_pos.shape[0], 'Par Count', count)
            index_list = df_pos.index
#             print(df_pos)
            df.loc[index_list[0], 'Parallel Clean Flag'] = count
            df.loc[flex_ind, 'Parallel Clean Flag'] = count
            df.loc[flex_ind, 'Clean_Start_Time'] = df_pos['Clean_Start_Time'][index_list[0]]
            df.loc[flex_ind, 'Clean_End_Time'] = df_pos['Clean_End_Time'][index_list[0]]
            df.loc[flex_ind, 'MC Group'] = df_pos['MC Group'][index_list[0]]
            df.loc[flex_ind, 'Constraint'] = df_pos['Constraint'][index_list[0]]
            df_flex = df[(df['Resource'].str.contains('Flex')) & (df['Parallel Clean Flag'].isnull())]
    return df, count

In [ ]:
def move_schedule_options(df = None, excl_res = [], mising_rec = None):
    clean_start = mising_rec['clean_start']
    last_possible = mising_rec['Last Possible Clean']

    df = cip_positions(df = df.copy(), var = 'MC Group')
    df = cip_positions(df = df.copy(), var = 'Constraint')

    df_move = df[(df['Clean_Start_Time']>=clean_start) & (df['Clean_Start_Time']<=last_possible) 
                 & (~df['Resource'].isin(excl_res)) 
                 & (df['Pre_Exist']!= 1) 
                 & (df['Parallel Clean Flag'].isnull())
                ]

    df_move['Cleaning Duration'] = df_move['Washing Time']*60
    df_move['Available Time'] = (df_move['Last Possible Clean'] - df_move['Clean_Start_Time'])/(np.timedelta64(1, 's')*60)

    df_move['MC Order'] = df_move['MC Group'].replace({'MC4 - Group': 1, 'MC1 - Group':1, 'MC3 - Group':3, 'MC2 - Group':3})
    df_move = df_move.sort_values(['MC Order', 'Available Time'], ascending = [True, False]) 
    return df_move

In [ ]:
def constrain_basedon_mc(mc_group = None):
    if (use_mc == 'MC4 - Group') | (use_mc == 'MC1 - Group'):
        useable_const = ['V', 'E']
    elif (use_mc == 'MC2 - Group') | (use_mc == 'MC3 - Group'):
        useable_const = ['F']
    return useable_const

In [ ]:
def final_constraint_use(constraint_list = [], df = None, move_clean_st = None):
    early_avail = []
    for const in constraint_list:
        df_last_used = df[(df['Constraint']==const) & (df['Clean_End_Time']<move_clean_start)]
        max_last_used = df_last_used['Clean_End_Time'].max()
        early_avail.append(max_last_used)
    last_const_used = min(early_avail)
    logical_min = early_avail.index(last_const_used)
    final_const = constraint_list[logical_min]
    return final_const, last_const_used

In [ ]:
def mc_schedule_moving(df = None, clean_start_move= None, move_res = None, mc = None):
    df_cipc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group',
                        sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)
    df_cipc = df_cipc[df_cipc['Available_Time']<0]
    
    move_add_time = abs(df_cipc.iloc[0]['Available_Time'])
    move_row_index = df[(df['MC Group']==mc) & 
                        (df['Clean_Start_Time'] == clean_start_move) &
                        (df['Resource']==move_res)].index[0]
    clean_end_cip = df_cipc.iloc[0]['Clean_End_Time']
    
    washing_time = df.loc[move_row_index]['Washing Time']
    clean_start_move = clean_end_cip + pd.Timedelta(minutes = 1)
    clean_end_move = clean_start_move + pd.Timedelta(minutes = washing_time*60)
    
    next_usage = df.loc[move_row_index]['Next_MC Group_Start']
    next_move_index = df[(df['MC Group']==mc) & 
                         (df['Clean_Start_Time'] == next_usage)].index[0]
    if df.loc[move_row_index]['Parallel Clean Flag'] >0:
        pc_num = df.loc[move_row_index]['Parallel Clean Flag']
        move_row_index = df[df['Parallel Clean Flag']==pc_num].index.tolist()
    df.loc[move_row_index, ['Clean_Start_Time', 'Clean_End_Time']] = [clean_start_move, clean_end_move]
    df = df.loc[:, :'New Last Possible']
    df = cip_positions(df = df.copy(), var = 'MC Group')
    df = cip_positions(df = df.copy(), var = 'Constraint')
    return df, next_move_index

In [ ]:
def const_schedule_moving(df = None):
    df_constc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
                      sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)
    df_constc = df_constc[df_constc['Available_Time']<0]
    move_constc = df_constc.iloc[0]['Constraint']
    rec_end = df_constc.iloc[0]['Clean_End_Time']
    move_mc = df_constc.iloc[0]['MC Group']
    rec_start = df_constc.iloc[0]['Clean_Start_Time']
#     rec_res = df[(df['Clean_End_Time']==rec_end) & (df['Clean_Start_Time']==rec_start) & 
#                  (df['MC Group']==move_mc) & (df['Constraint']==move_constc)].iloc[0]['Resource']
    
    move_add_time = abs(df_constc.iloc[0]['Available_Time'])
#     move_row_index = df[(df['Constraint']==move_constc) & 
#                         (df['Last_Constraint_End'] == rec_end) & 
#                         (df['Resource'] != rec_res)].index[0]
    df_rec = df[(df['Clean_End_Time']==rec_end) & (df['Clean_Start_Time']==rec_start) & 
                 (df['MC Group']==move_mc) & (df['Constraint']==move_constc)]
    rec_res = df_rec.iloc[0]['Resource']
    rec_index = df_rec.index[0]
    move_row_df = df[(df['Constraint']==move_constc) & (df['Clean_Start_Time']== rec_end - pd.Timedelta(minutes = move_add_time))]
    move_row_index = [i for i in move_row_df.index if i != rec_index][0]
    clean_end_cip = rec_end
    washing_time = df.loc[move_row_index]['Washing Time']
    clean_start_move = clean_end_cip + pd.Timedelta(minutes = 1)
    clean_end_move = clean_start_move + pd.Timedelta(minutes = washing_time*60)

#     next_usage = df.loc[move_row_index]['Next_Constraint_Start']
#     next_move_index = df[(df['Constraint']==move_constc) & 
#                          (df['Clean_Start_Time'] == next_usage)].index[0]
    if df.loc[move_row_index]['Parallel Clean Flag'] > 0:
        pc_num = df.loc[move_row_index]['Parallel Clean Flag']
        move_row_index = df[df['Parallel Clean Flag']==pc_num].index.tolist()

    df.loc[move_row_index, ['Clean_Start_Time', 'Clean_End_Time']] = [clean_start_move, clean_end_move]
    df = df.loc[:, :'New Last Possible']
    df = cip_positions(df = df.copy(), var = 'MC Group')
    df = cip_positions(df = df.copy(), var = 'Constraint')
    return df

# Creating df_const_avail and df_cip_avail
1. Sort value of data_bkp on basis of Priority, Resource Rank, Usage_End and store it in data.
- This will help us to sort our resources, higher priority will appear first
2. From df_cip_gconst_bkp drop slno as not in use.
3. Rename column Value to Constraint in df_cip_gconst
4. Create a blank dataframe final_df which columns from data dataframe
5. min_clean = data.usage_end.min()
- min start time of cleaning
6. max_clean = max of clean end time
7. max_dht_clean = take the max DHT of the record where clean_end = max_clean
- As the cleaning can be scheduled withing the maxt DHT available for last utilization, dht is taken into consideration
8. 15 hours is added to max_clean to have a safe bandwidth of max_clean
9. Static time is just for reference
10. df_cip_avail -- dataframe containing unique resources
11. df_const_avail --- contains unique constraints
12. df_cip_avail, df_const_avail dataframes will contain Clean_Start_Time, Clean_End_Time, idle_till_time and Available_Time
- Clean_start_time initially will contain max_clean values
- Clean_End_Time = min_clean
- Available_Time = max_clean - min_clean
- this dataframes will get updated after each allocation of cleaning to utilizations.


# visit later
new record inserted:
new record ideal time = previous record clean start time
new record clean start time = utilization's clean start time
new record clean end time = utilization's clean end time


DOUBT
what about the clean end time we kept earlier dont we require that ideal time??

In [ ]:
# Creation of blanl lists to be used while checking the constraints
# data = data_bkp.sort_values(['Priority', 'Usage_End']).copy()
data = data_bkp.sort_values(['Priority', 'Resource Rank', 'Usage_End']).reset_index(drop = True).copy()
df_cip_gconst = df_cip_gconst_bkp.drop('slno', axis = 1).drop_duplicates().copy()
df_cip_gconst = df_cip_gconst.rename(columns = {'value': 'Constraint'})
final_df = pd.DataFrame(columns = data.columns)

# Some blank lists might be used in the process
used_res = []; used_const1 = []; used_mc1 = []; mc_avail = []

In [ ]:
################# change 06/12/21

In [ ]:
min_clean = min(data.Clean_Start_Time.min(),data.Usage_Start.min())
if len(df_qcip)==0:
    min_clean = data.Usage_Start.min()
else:
    min_clean = min(data.Clean_Start_Time.min(),data.Usage_Start.min())

In [ ]:
#min_clean = data.Usage_Start.min()

In [ ]:
################# change 06/12/21 end

In [ ]:
#min_clean=  pd.to_datetime('05/05/2022 15:48:00', format = '%d/%m/%Y %H:%M:%S')
max_clean = data.clean_end.max()

In [ ]:
# Cleaning to be given
# min_clean = data.Usage_Start.min()
#min_clean = data.Usage_Start.min()
max_clean = data.clean_end.max()

max_dht_clean = data[data['clean_end'] == max_clean].reset_index(drop = True)['Max DHT'][0]

max_clean = max_clean + pd.Timedelta(minutes=15*60)


# min_cht_clean = data[data['Usage_Start']==min_clean].iloc[0]['Max CHT']
# min_clean = min_clean - pd.Timedelta(minutes=min_cht_clean*60)

static_date = np.datetime64('2000-01-01'); count = 0

df_cip_avail = df_cip_gconst[['MC Group']].drop_duplicates()
df_const_avail = df_cip_gconst[['Constraint']].drop_duplicates()
df_cip_avail['Clean_Start_Time']= df_const_avail['Clean_Start_Time'] = max_clean
df_cip_avail['Clean_End_Time'] = df_const_avail['Clean_End_Time'] = min_clean
df_cip_avail['idle_till_time'] = df_const_avail['idle_till_time']=  max_clean
df_cip_avail['Available_Time'] = (max_clean - min_clean)/(np.timedelta64(1, 's')*3600)
df_const_avail['Available_Time'] = (max_clean - min_clean)/(np.timedelta64(1, 's')*3600)
# df_cip_avail['Last_pos_Clean'] = df_cip_avail['idle_till_time'] - pd.Timedelta(minutes=clean_dur*60+10)

#######new change start 3/12/2021

In [ ]:
# df_downtime_test = df_downtime.copy()
#df_downtime_test['Clean_Start_Time'] = pd.to_datetime(df_downtime_test['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
#df_downtime_test['Clean_End_Time'] = pd.to_datetime(df_downtime_test['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
correct_df =df_downtime[df_downtime['Clean_End_Time']< max_clean]
incorrect_df = df_downtime[df_downtime['Clean_End_Time']> max_clean]
incorrect_df['Clean_End_Time']=max_clean
df_downtime = pd.concat([correct_df,incorrect_df])
# df_downtime_test['Clean_End_Time'] =  np.where(df_downtime_test['Clean_End_Time']> max_clean , max_clean , pd.to_datetime(df_downtime_test['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S'))

#for index , row in df_downtime_test.iterrows():




##########new change end 3/12/2021

# PC flag
1. Create a column pc_flag in data dataframe containing 1 if resourse is present in par_cl_res else 0
- this flag is to indentify ulizations/resources which can have a parallel cleaning.
2. data_par dataframe contains prallel resources where MC group is null.
- This will be used in parallel cleaning optimization
3. create data_flex data frame containing all flexible resources
4. Remove these from data dataframe
5. append the flexible resources @ end

In [ ]:
data['pc_flag'] = np.where(data['Resource'].isin(par_cl_res), 1, 0)

col_order = ['Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint', 'Cleaning Type']
dataA = data[col_order]
dataB = data.drop(col_order, axis = 1)
data = pd.concat([dataB, dataA], axis = 1)

data_par = data[(data['pc_flag']==1)  & (data['MC Group'].isnull())].sort_values('Usage_End').reset_index(drop = True)
data_flex = data[data["Resource"].str.startswith('Flexible')].sort_values('Usage_End').reset_index(drop = True)
data = data[~data["Resource"].str.startswith('Flexible')].reset_index(drop = True)
data = pd.concat([data, data_flex]).reset_index(drop = True)


# Recleanings and final df
1. Take all freezed records in data_freeze dataframe
2. call include_recleans function to add rows of expected recleanings
3. Sort the data with clean start, last possible clean and rank
4. place the flexible data at the end
5. pre_exit = in case allocation is done will be indicated as 1 else 0
- all freezed records will hold 1
6. Initial assignment and push assignment is nan for all records
7. final df contains all assigned cleanings

In [ ]:
data_freeze = data[~data['Cleaning Type'].isnull()]
data = include_recleans(df= data.copy())
data = data.sort_values(['clean_start', 'Last Possible Clean', 'Resource Rank']).reset_index(drop = True).copy()
data_flex = data[data['Resource'].str.contains('Flexible')]
data_other = data[~data['Resource'].str.contains('Flexible')]

data = pd.concat([data_other, data_flex]).reset_index(drop = True)

data['Pre_Exist'] = np.where(data['MC Group'].isnull(), 0, 1)
data['Initial Assignment'] = np.nan
data['Push Assignment'] = np.nan
# data['Parallel Clean Flag'] = np.nan

final_df = data[data['Pre_Exist'] ==1].reset_index(drop = True)
data = pd.concat([final_df, data[data['Pre_Exist'] != 1]]).reset_index(drop = True)
data = pd.concat([data['Resource'], data.drop('Resource', axis = 1)], axis = 1)
# final_df['MC Group'] = final_df['MC Group']+ ' - Group'

In [ ]:
data.to_clipboard(index = False)

In [ ]:
count = 0; par_count = last_par_count
data_freeze['Cleaning Type'].value_counts()

In [ ]:
data.to_clipboard()

In [ ]:
for index, row in data.iterrows():
    res = row['Resource']; clean_start = row['clean_start']; clean_end = row['clean_end']
    max_clean_start = row['Last Possible Clean']; priority =  row['Priority']; washing_time = row['Washing Time']
    
    if final_df[(row['LOT'] == final_df['LOT']) & (res == final_df['Resource']) & 
                (row['Type'] == final_df['Type']) & 
                (row['pc_flag'] ==1) & (final_df['Cleaning Type'].isnull())].shape[0] > 0:
        continue
    if row['Pre_Exist']==1:
        continue

    mc_list = df_cip_gconst[df_cip_gconst['Group'].isin([1,2])]['MC Group'].unique().tolist()
    const_list = df_cip_gconst[df_cip_gconst['Group'].isin([1,2])]['Constraint'].unique().tolist()
        
    df_cip_avail_new = cleaning_availablity(df = final_df.copy(), sort_by_col = 'Clean_Start_Time', 
                                            sub_col =  'Clean_End_Time', for_col = 'MC Group', max_clean = max_clean, 
                                            clean_dur = washing_time, df_avail = df_cip_avail.copy(), buffer_mins = buffer)
    print('index:', index, res, row['LOT'], 'df Shape:', final_df.shape)
    print('Missing Rec',final_df[final_df['MC Group'].isnull()].shape)
    print('MC Overlap:', df_cip_avail_new[df_cip_avail_new['Available_Time']<0].shape[0])

    df_const_avail_new = cleaning_availablity(df = final_df.copy(), sort_by_col = 'Clean_Start_Time', 
                                              sub_col =  'Clean_End_Time', for_col = 'Constraint', max_clean = max_clean, 
                                              clean_dur = washing_time, df_avail = df_const_avail.copy(),buffer_mins = 0)
    
    print('Const Overlap:', df_const_avail_new[df_const_avail_new['Available_Time']<0].shape[0])
    print('DHT Violation:', final_df[final_df['Last Possible Clean']< final_df['Clean_Start_Time']].shape[0])
    df_m = pd.DataFrame()
    pc_option = df_par_clean[df_par_clean['Process resource']==res]['PC Option'].tolist()
    pc_av_logic =  data_par['Resource'].isin(pc_option).sum()
#     if index == 62:
#         break
    for mins in range(0,1000):
        clean_start = row['clean_start'] + pd.Timedelta(minutes=mins)
        clean_end = clean_start + pd.Timedelta(minutes= washing_time*60)
        if (max_clean_start < clean_start) & (df_m.shape[0] ==0):
            final_df = pd.concat([final_df, data.loc[[index],:]])
            print('No Slot Available for:', res, clean_start)
            count = count +1
            rows=1
            dict_values = {'Clean_Start_Time': np.nan, 'Clean_End_Time': np.nan, 
                           'MC Group': np.nan, 'Constraint': np.nan, 'Usage_Start': None, 
                           'Usage_End': None, 'Maint_Flag': 0}
            break
        elif (max_clean_start < clean_start):
            break
        mc_possible_df = check_cond(df = df_cip_avail_new, col = 'MC Group', cleaning_start = clean_start, 
                                 cleaning_end = clean_end)
        mc_possible = mc_possible_df['MC Group'].unique().tolist()
        const_possible_df = check_cond(df = df_const_avail_new, col = 'Constraint', cleaning_start = clean_start, 
                                    cleaning_end = clean_end)
        const_possible = const_possible_df['Constraint'].unique().tolist()

        
        if (len(mc_possible)==0) | (len(const_possible)==0):
            continue
        mc_possible = [elem for elem in mc_possible if elem in mc_list]
        const_possible = [elem for elem in const_possible if elem in const_list]
        
        
        th_time = max_clean_start
        reclean = 0
        rows = 0
        dict_values = {}
        for mc in mc_possible:
            for const in const_possible:
                rows = df_cip_gconst[(df_cip_gconst['Resource'] ==res) & (df_cip_gconst['Constraint'] ==const) & 
                                     (df_cip_gconst['MC Group'] ==mc)].shape[0]
                
                if rows>0:
                    cl_start, cl_end = enough_gap_constraint(df = final_df.copy(), mc_group = mc, resource = res, 
                                                             mins = buffer,clean_start_tm = clean_start,
                                                             clean_end_tm = clean_end, th_time = max_clean_start)
                    cl_start, cl_end = test_overlap(df = data.copy(), fin_data= final_df.iloc[:-1,:], cl_st = cl_start, 
                                                    cl_end = cl_end, cip_gr = mc,const_gr= const, mins = buffer)
                    if (cl_start > th_time):
                        rows = 0
                        continue
                    clean_start, clean_end = cl_start, cl_end
                    break
                else:
                    continue
            if rows>0:
                dict_values = {'Clean_Start_Time': [clean_start], 'Clean_End_Time': [clean_end], 'MC Group': [mc], 
                               'Constraint': [const], 'Initial Assignment': [1], 'Usage_Start': [None], 
                               'Usage_End': [None], 'Maint_Flag': [0]}
                break
            else:
                continue

        if (rows>0):
            df_t = pd.DataFrame(dict_values)
            df_m = pd.concat([df_m, df_t]).reset_index(drop = True)
            if (pc_av_logic > 0) & (row['pc_flag'] ==1) :
                continue
            else:
                break
        # Extra added step
        else:
            continue   

    if df_m.shape[0]>0:
        dict_values = df_m.to_dict(orient='records')[0]
        mc = dict_values['MC Group']; const = dict_values['Constraint']
        if (row['pc_flag'] != 1) | (pc_av_logic == 0):
            df_cip_avail['Clean_End_Time'].loc[(df_cip_avail['MC Group']==mc)] = max_clean
            df_const_avail['Clean_End_Time'].loc[(df_const_avail['Constraint']==const)] = max_clean
        data = CIP_assignment(df = data, index = index, main_flag = 0, val_dict = dict_values)
        temp_data1 = data.loc[[index],:].copy()
        final_df = pd.concat([final_df, temp_data1])    
    if (max_clean_start < clean_start) & (df_m.shape[0] ==0):
#         break
        try:
            final_df_push = Push_Cleaning(df = final_df.copy(), clean_start = row['clean_start'], clean_end = row['clean_end'], 
                                         washing_time = washing_time, usage_end = row['Usage_End'], usage_start = row['Usage_Start'],
                                         index_val = index, last_pos = row['Last Possible Clean'])
            
        except Exception as e:
            print(e)
            print('Module****Pushing Skipped****')
            continue
        final_df = final_df_push.reset_index(drop = True).copy()
        final_df.loc[index, 'Push Assignment'] = 1
        temp_data1 = final_df_push.tail(1)
        mc =  temp_data1['MC Group'].tolist()[0]; const =  temp_data1['Constraint'].tolist()[0]
        clean_start =  temp_data1['Clean_Start_Time'].tolist()[0]
        clean_end =  temp_data1['Clean_End_Time'].tolist()[0]
        dict_values = {'Clean_Start_Time': [clean_start], 'Clean_End_Time': [clean_end], 'MC Group': [mc], 
                        'Constraint': [const], 'Push Assignment': [1], 'Usage_Start': [None], 'Usage_End': [None], 
                        'Maint_Flag': [0]}
        df_m = pd.DataFrame(dict_values) 

    if (pc_av_logic > 0) & (row['pc_flag'] ==1):
#         break
        last_record = final_df.iloc[[-1], :]
        for i in range(0, df_m.shape[0]):
            final_df = final_df.iloc[:-1,:]
            dict_values = df_m.to_dict(orient='records')[i]
            data = CIP_assignment(df = data, index = index, main_flag = 0, val_dict = dict_values)
            temp_data1 = data.loc[[index],:].copy()
            final_df = pd.concat([final_df, temp_data1])
            df_cip_av = cleaning_availablity(df = final_df.copy(), sort_by_col = 'Clean_Start_Time', 
                                            sub_col =  'Clean_End_Time', for_col = 'MC Group', max_clean = max_clean, 
                                            clean_dur = washing_time, df_avail = df_cip_avail.copy(), buffer_mins = buffer)

            df_const_av = cleaning_availablity(df = final_df.copy(), sort_by_col = 'Clean_Start_Time', 
                                                    sub_col =  'Clean_End_Time', for_col = 'Constraint', max_clean = max_clean, 
                                                    clean_dur = washing_time, df_avail = df_const_avail.copy(), buffer_mins = 0)
            mc = dict_values['MC Group']; const = dict_values['Constraint']
            clean_start = dict_values['Clean_Start_Time']; clean_end = dict_values['Clean_End_Time']

            mc_data = df_cip_av[(df_cip_av['Clean_Start_Time'] == clean_start) & (df_cip_av['MC Group'] == mc) & 
                                (df_cip_av['Clean_End_Time'] == clean_end)]
            mc_data = mc_data.reset_index(drop = True)

            const_data = df_const_av[(df_const_av['Clean_Start_Time'] == clean_start) & (df_const_av['Constraint'] == const) & 
                                     (df_const_av['Clean_End_Time'] == clean_end)]
            const_data = const_data.reset_index(drop = True)
            ###### ##### ####### ####### ########
            df_senstive = data[(clean_start>= data['clean_start']) & (data['MC Group'].isnull()) & 
                                   (data['idle_time'].round(1)<=1)]
            ######  #######  #######  ##########
            temp_data1['New Last Possible'] = min(mc_data['Last_pos_Clean'][0],const_data['Last_pos_Clean'][0], row['Last Possible Clean'])
            clean_av = min(mc_data['Last_pos_Clean'][0],const_data['Last_pos_Clean'][0], row['Last Possible Clean'])

            pc_option = df_par_clean[df_par_clean['Process resource']==res]['PC Option'].tolist()
            temp_data2 = data_par[(data_par['clean_start'] < clean_av) & (data_par['Usage_End'] < clean_start) &
                          (data_par['Last Possible Clean'] >= clean_start) & (data_par['Resource'].isin(pc_option))]
            
            if (temp_data2.shape[0]>0) & (df_senstive.shape[0]==0):
                temp_data2=temp_data2.sort_values('Unutilized_gap').head(1)
                drop_index = temp_data2.index[0]
                
                par_count = par_count + 1
                data_par = data_par.drop(drop_index, axis = 0)
                print('Hurrey...! Its Parellal No:', par_count)
                break
            elif df_senstive.shape[0]>0:
                temp_data2 = pd.DataFrame(columns = temp_data1.columns)
                break
            else:
                continue
        if temp_data2.shape[0]==0:
            print('Saste me kam chalalo')
            dict_values = df_m.to_dict(orient='records')[0]
            data = CIP_assignment(df = data, index = index, main_flag = 0, val_dict = dict_values)
            temp_data1 = last_record.copy()
        temp_data = pd.concat([temp_data1, temp_data2.sort_values('Unutilized_gap').head(1)])
        temp_data[['MC Group', 'Constraint']] = temp_data[['MC Group', 'Constraint']].ffill()
        if temp_data.shape[0]> 1:
            par_clean_start = temp_data2.reset_index(drop = True)['clean_start'][0]
            par_clean_start = max(par_clean_start, clean_start)
            temp_data['Clean_Start_Time'] = par_clean_start
            temp_data['Clean_End_Time'] = temp_data['Clean_Start_Time'] + pd.to_timedelta(temp_data['Washing Time']*3600, 
                                                                                          unit='s')
            temp_data['Parallel Clean Flag'] = par_count
        temp_data1 = temp_data1.iloc[0]
        data_par  = data_par[~((data_par['LOT']== temp_data1['LOT']) & (data_par['Type'] == temp_data1['Type']) & 
                            (data_par['Resource'] == temp_data1['Resource']))]
        print('Data Parallel Shape', data_par.shape)
        mc = temp_data['MC Group'].iloc[0]
        const  = temp_data['Constraint'].iloc[0]
        df_cip_avail['Clean_End_Time'].loc[(df_cip_avail['MC Group']==mc)] = max_clean
        df_const_avail['Clean_End_Time'].loc[(df_const_avail['Constraint']==const)] = max_clean
        final_df = pd.concat([final_df.iloc[:-1,:], temp_data]).reset_index(drop = True)

In [ ]:
final_df.to_clipboard()

In [ ]:
#final_df.to_clipboard()

In [ ]:
final_df_bkp = final_df.reset_index(drop = True).copy()
par_count_bkp = par_count

In [ ]:
par_count = par_count_bkp
final_df = final_df_bkp.copy()

In [ ]:
df = final_df.reset_index(drop = True).copy()

df_cipc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
              sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

df_constc = cip_avail(df = df.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
              sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

const_overlap = df_constc[df_constc['Available_Time']<0].shape[0]
dht_violation = df[df['Last Possible Clean']< df['Clean_Start_Time']].shape[0]
mc_overlap = df_cipc[df_cipc['Available_Time']<0].shape[0]

const_overlap + dht_violation + mc_overlap

In [ ]:
final_df, par_count = process_missing_parallel(df= final_df.copy(), count=par_count)

#### One step Automated Miss Clean

In [ ]:
final_df.loc[(final_df['MC Group'].isnull()) & (~final_df['Resource'].str.contains('Flex')), ['MC Group', 'Constraint']] = 'NA'

In [ ]:
exclude_schedules = ['P1', 'P2','P3', 'P4', 'AS26 TD', 'SA', 'SB', 'SC', 'SD']
final_df = final_df.reset_index(drop = True).copy()
num_mis_rec = final_df['MC Group'].isnull().sum()
for im in range(num_mis_rec):
    df_mis_cleaning = final_df[final_df['MC Group'].isnull()]
    print('Missing Shape', df_mis_cleaning.shape)
    if df_mis_cleaning.shape[0]==0:
        break
    mis_rec = df_mis_cleaning.iloc[0]
    mis_index = df_mis_cleaning.index[0]
    df_backup = final_df.copy()
    clean_start = mis_rec['clean_start']; clean_end = mis_rec['clean_start']
    df_move_options = move_schedule_options(df = final_df, excl_res = exclude_schedules, mising_rec = mis_rec)
    if df_move_options.shape[0] == 0:
        const_overlap = 0; dht_violation = 0; mc_overlap = 0
        final_df.loc[mis_index, 'MC Group'] = 'NA'
        final_df.loc[mis_index, 'Constraint'] = 'NA'
    print('Move Options' , df_move_options.shape[0])
    for i in df_move_options.index:
        df = df_backup.copy(); move_row_index = i;
        move_rec = df_move_options.loc[i]; move_const = move_rec['Constraint']; use_mc = move_rec['MC Group']; 
        move_clean_start = move_rec['Clean_Start_Time']; last_mc_used = move_rec['Last_MC Group_End']
        last_possible_clean = move_rec['Last Possible Clean']; moved_res = move_rec['Resource']

        use_const = constrain_basedon_mc(mc_group = None)
        const_detail = final_constraint_use(constraint_list = use_const, 
                                            df = df.copy(), move_clean_st = move_clean_start)
        use_const = const_detail[0]; last_const_used = const_detail[1]

        assign_clean_start = max(last_mc_used, last_const_used, clean_start)+pd.Timedelta(minutes = 1)
        assign_clean_end = assign_clean_start+pd.Timedelta(minutes = mis_rec['Washing Time']*60)
        if last_possible_clean < assign_clean_start:
            print('Initial start time crossing the threshold time')
            continue
        df.loc[mis_index, ['Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint']] = [assign_clean_start, 
                                                                                           assign_clean_end, use_mc, use_const]
        clean_start_move = move_rec['Clean_Start_Time']

        df = df.loc[:, :'New Last Possible']
        df = cip_positions(df = df.copy(), var = 'MC Group')
        df = cip_positions(df = df.copy(), var = 'Constraint')

        const_overlap, dht_violation, mc_overlap = check_overlaps(df = df.copy())
        print('Initial Slot assignment', const_overlap, dht_violation, mc_overlap)
        if (dht_violation==0) & (mc_overlap==0) & (const_overlap==0):
            print('1-No need to work on Pushing anything, Assigning to Final_df')
            df = df.loc[:, :'New Last Possible']
            final_df = df.copy()
            final_df, par_count = process_missing_parallel(df= final_df.copy(), count=par_count)
            break
        elif (mc_overlap>0):
            updated_df = df.copy()
            mc_loop_count = 0
            while (dht_violation ==0) & (mc_overlap>0):
                mc_loop_count = mc_loop_count+1
                updated_df, next_index = mc_schedule_moving(df = updated_df.copy(), clean_start_move= move_clean_start, 
                                                            move_res = moved_res, mc = use_mc)
                moved_res = updated_df.loc[next_index]['Resource']
                move_clean_start = updated_df.loc[next_index]['Clean_Start_Time']
                move_constc = updated_df.loc[next_index]['Constraint']
                const_overlap, dht_violation, mc_overlap = check_overlaps(df = updated_df.copy())
                print('MC Level Push', const_overlap, dht_violation, mc_overlap)
                if (dht_violation ==0) & (mc_overlap==0) & (const_overlap==0):
                    print('2-No need to work on Constrainint, Assigning to Final_df')
                    updated_df = updated_df.loc[:, :'New Last Possible']; break
                elif (mc_overlap==0) & (const_overlap>0):
                    print('Check with constraint pushing'); break
                elif mc_overlap>2:
                    print('Go to next Move option'); break
                if mc_loop_count==50:
                    break
            if (dht_violation>0) | (mc_overlap>=2):
                print('Due to more than 2 overlaps Go to next move option'); continue
            elif (dht_violation==0) & (mc_overlap==0) & (const_overlap>0):
                const_loop_count = 0
                while (dht_violation==0) & (mc_overlap<=1) & (const_overlap>0):
                    updated_df = const_schedule_moving(df = updated_df.copy())
                    const_overlap, dht_violation, mc_overlap = check_overlaps(df = updated_df.copy())
                    print('Constraint Level Push', const_overlap, dht_violation, mc_overlap)
                    if (dht_violation==0) & (mc_overlap==0) & (const_overlap==0):
                        print('3-Constrainint push successful, Assigning to Final_df')
                        updated_df = updated_df.loc[:, :'New Last Possible']; break
                    elif (dht_violation>0) | (mc_overlap>1):
                        break
                    if const_loop_count == 50:
                        break
        if (dht_violation==0) & (mc_overlap==0) & (const_overlap==0):
            final_df = updated_df.copy()
            final_df, par_count = process_missing_parallel(df= final_df.copy(), count=par_count)
            print('Successfull Processed')
            df_mis_cleaning = df_mis_cleaning.iloc[1:,:]
            break
        else:
            print('Go to next record')
            continue
    if (dht_violation!=0) | (mc_overlap!=0) | (const_overlap !=0):
        if (i == df_move_options.index[-1]):
            final_df.loc[mis_index, 'MC Group'] = 'NA'
            final_df.loc[mis_index, 'Constraint'] = 'NA'
            print('<<<<<<<<bhai>>>>>>>>')
    #         df_mis_cleaning = df_mis_cleaning.iloc[1:,:]
            continue
final_df.loc[final_df['MC Group']=='NA', ['MC Group', 'Constraint']] = np.nan

### Maintenance

In [ ]:
final_df_bkp = final_df.reset_index(drop = True).copy()

In [ ]:
final_P_assigned = final_df_bkp.reset_index(drop = True).copy()

In [ ]:
final_df = final_df_bkp.copy()
final_df['Clean_Start_Time'] = pd.to_datetime(final_df['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
final_df['Clean_End_Time'] = pd.to_datetime(final_df['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
final_df.loc[final_df['Cleaning Type']=='Reclean', 'Next Usage'] = final_df['Usage_Start']
# final_df['Last_M_endtime'] = final_df['Next Usage'] - pd.to_timedelta(final_df['Washing Time']*3600, unit='s') #subtract 30 minutes
# final_df['Last_M_endtime'] = final_df['Last_M_endtime'].fillna(final_df['Usage_End'].max())
final_df['Last_M_endtime'] = final_df['Last Possible Clean']
final_df['Time_av_for_M'] = final_df['Last_M_endtime'] - final_df['Clean_End_Time']
final_df['Time_av_for_M'] = final_df['Time_av_for_M']/(np.timedelta64(1, 's')*3600)

In [ ]:
df = final_df.copy()
x_df_remaining = df_cip_gconst.copy()
count = 0
for index, row in df_maintenance.iterrows():
    count = count + 1
    res = row['Resource']; start_time = row['Start_time']; end_time = row['End_time']; M_time = row['M_time']
    M_DHT = row['Max DHT']; M_code = row['Code']
    nearest_df = df[df['Resource']==res]
    nearest_df['Maintain_pre_CHT'] = start_time - pd.to_timedelta(nearest_df['Max CHT']*3600, unit='s')
    nearest_df = nearest_df[(nearest_df['Maintain_pre_CHT'] < nearest_df['Clean_End_Time']) & 
                            (nearest_df['Time_av_for_M'] > M_time) & (nearest_df['Cleaning Type'] != 'q-Pre Cleaning') ]
    washingtime = mapping_df[mapping_df['Resource'] == res].reset_index(drop = True)['Washing Time'][0]
    print(res, nearest_df.shape)
    ######################## change on 01/12/2021
    map_columns = ['MC Group', 'Constraint', 'Washing Time','Clean_Start_Time', 'Clean_End_Time','Parallel Clean Flag']
    ####################### change on 01/12/2021
    df_mc_group = df[map_columns]
    
    df_MC_available = cleaning_availablity(df = df_mc_group.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
                                           sub_col = 'Clean_End_Time', max_clean = max_clean, clean_dur = washing_time, 
                                           buffer_mins = buffer/5)                                                              #why buffer/5?
    df_cons_available = cleaning_availablity(df = df_mc_group.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
                                             sub_col = 'Clean_End_Time', max_clean = max_clean, clean_dur = washing_time, 
                                             buffer_mins = 0)
    rows = 0
    if nearest_df.shape[0]>0:
        for nearindex , nearrow in nearest_df.iterrows():
            print(nearindex)
            nearres = nearrow['Resource']; clean_end = nearrow['Clean_End_Time']; clean_start = nearrow['Clean_Start_Time']
            last_possible = nearrow['Last_M_endtime']; washingtime = nearrow['Washing Time']
            
            start_time = clean_end + pd.Timedelta(minutes = buffer/2)
            end_time = start_time + pd.Timedelta(minutes = M_time*60)
            print(start_time, end_time)
            if start_time < clean_end:
                continue 
            for mins in range(0,1000):
                clean_assign_start = end_time + pd.Timedelta(minutes = mins)
                clean_assign_end = clean_assign_start + pd.Timedelta(minutes = washingtime*60)
                end_time = clean_assign_start - pd.Timedelta(minutes = 1)
                start_time = end_time - pd.Timedelta(minutes = M_time*60)
                if clean_assign_start > last_possible:
                    print(nearres, ' No Slot for cleaning')
                    break
                rows, nearest_df = clean_allocate(df_mc = df_MC_available, df_cons = df_cons_available, res = nearres,
                                                  df_update = df.copy(), index = nearindex, loop_df = nearest_df,main_flag = 1,
                                                  mc_const_df = df_cip_gconst.copy(), clean_start = clean_assign_start, 
                                                  clean_end = clean_assign_end, m_start = start_time, m_end = end_time, 
                                                  th_time = last_possible, buffer_time= buffer/5)
                if rows>0:
                    nearest_df.loc[nearindex,'run_flag'] = 0
                    nearest_df.loc[nearindex,'LOT'] = M_code
                    df = pd.concat([df, nearest_df.loc[[nearindex],:]])
                    print('Final', start_time, end_time)
                    break
            if rows>0:
                break
    if (nearest_df.shape[0]==0) | (rows==0):
        print('Me Andar aya')
        blank_df = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns)
        ########################## new section added on 03/12/2021
        #if df[df['Resource']==res].shape[0] == 0:
        #    print("I didn't had any utilization")
        #    start_time = start_time
        #    end_time = start_time + pd.Timedelta(minutes = M_time*60)
        #    blank_df = blank_df
        #    print(start_time, end_time)
        #    if len(df_cip_gconst[df_cip_gconst["Resource"]==res])==0:
        #        add_mapping= mapping_df[mapping_df["Resource"]==res]
        #        select_var = ['Resource']+allmcgroup
        #        add_mapping=add_mapping.loc[:, select_var]
        #        add_mapping = pd.melt(add_mapping, id_vars = add_mapping.columns[0], value_vars=add_mapping.columns[1:], var_name='MC Group')
        #        add_mapping = add_mapping[~(add_mapping['value'].isnull())]
        #        add_mapping = add_mapping[~(add_mapping['MC Group'].isnull())]
        #        add_mapping.columns = ["Resource","MC Group","Constraint"]
        #        df_cip_gconst = pd.concat([df_cip_gconst,add_mapping]).reset_index(drop = True)  
        ########################## new section ended on 03/12/2021
        if df[df['Resource']==res].shape[0] > 0:
            start_time = df[df['Resource']==res]['Clean_End_Time'].max() + pd.Timedelta(minutes = buffer/5)
            end_time = start_time + pd.Timedelta(minutes = M_time*60)
            blank_df = df[df['Resource']==res].head(1).reset_index(drop = True)
            print(start_time, end_time)
        for mins in range(0,1200):
#             print(mins)
            clean_assign_start = end_time + pd.Timedelta(minutes = mins*1)
            clean_assign_end = clean_assign_start + pd.Timedelta(minutes = washingtime*60)
            start_time = start_time + pd.Timedelta(minutes = mins*1)
            end_time = end_time + pd.Timedelta(minutes = mins*1)
            
            rows, blank_df = clean_allocate(df_mc = df_MC_available, df_cons = df_cons_available, res = res,
                                            df_update = df.copy(), index = 0, loop_df = blank_df, main_flag = 1, 
                                            mc_const_df = df_cip_gconst.copy(), clean_start = clean_assign_start, 
                                            clean_end = clean_assign_end, m_start = start_time, m_end = end_time, 
                                            th_time = max_clean, buffer_time= buffer/5)
                   
            if rows>0:
                print('Yes Mil gaya')
                clean_assign_start = blank_df['Clean_Start_Time'][0]
                end_time = clean_assign_start - pd.Timedelta(minutes=10)
                start_time = end_time -  pd.Timedelta(minutes=M_time*60)
                
                blank_df.loc[0,'Usage_Start'] = start_time
                blank_df.loc[0,'Usage_End'] = end_time
                blank_df.loc[0,'clean_flag'] = 1
                blank_df.loc[0,'run_flag'] = 0
                blank_df.loc[0,'Resource'] = res
                blank_df.loc[0,'LOT'] = M_code
                #if df[df['Resource']==res].shape[0] == 0:
                #    print("aya"+ str(df[df['Resource']==res].shape[0]))
                #    blank_df.loc[0,'Last Possible Clean'] = max_clean 
                df = pd.concat([df, blank_df.loc[[0],:]])
                break

In [ ]:
df = df[[x for x in df.columns if x not in ['']]]
#col_null = ['Utilized', 'addT', 'Next Usage', 'Unutilized_gap', 'clean_start', 'clean_end', 'idle_time',
#       'run_flag', 'clean_flag', 'Max DHT Time', 'Last Possible Clean',
#       'Last Cleaning_DHT', 'Start Cleaning_CHT', 'Last_pos_preclean',
#       'Day Flag', 'pc_flag','Initial Assignment', 'Cleaning Type',
#       'Push Assignment', 'Parallel Clean Flag', 'New Last Possible', 'Pre_Exist', 'Cleaning Type']
###############change on 1/12/2021
col_null = ['Utilized', 'addT', 'Next Usage', 'Unutilized_gap', 'clean_start', 'clean_end', 'idle_time',
       'run_flag', 'clean_flag', 'Max DHT Time', 'Last Possible Clean',
       'Last Cleaning_DHT', 'Start Cleaning_CHT', 'Last_pos_preclean',
       'Day Flag', 'pc_flag','Initial Assignment', 'Cleaning Type',
       'Push Assignment', 'Parallel Clean Flag', 'Pre_Exist', 'Cleaning Type']
############# chnage on 1/12/2021 end

if df_maintenance.shape[0] >0:
    df.loc[df['Maint_Flag']==1, col_null]= np.nan
else:
    df['Maint_Flag'] = np.nan
df_rec_performed = df[~df['Cleaning Type'].isnull()].reset_index(drop = True)
df = df[df['Cleaning Type'].isnull()].reset_index(drop = True)
    

In [ ]:
final_M_assigned = df.reset_index(drop = True)
# Creation of cleaning start just after the first buffer is finished
df['clean_start'] = df['Usage_End'] + pd.Timedelta(minutes = 1)

# Assuming the cleaning time is exactly as the defined time
df['clean_end'] = np.nan

temp = mapping_df.rename(columns = {'Max DHT': 'DHT Time', 'Max CHT': 'CHT Time'})
df_assigned = pd.merge(df.copy(), temp[['Resource', 'DHT Time', 'CHT Time']].drop_duplicates(), 
                       on = 'Resource', how = 'inner')

df_assigned['Max DHT'] = np.where(df_assigned['Max DHT'].isnull(), 
                                          df_assigned['DHT Time'], df_assigned['Max DHT'])
df_assigned['Max CHT'] = np.where(df_assigned['Max CHT'].isnull(), 
                                          df_assigned['CHT Time'], df_assigned['Max CHT'])
df_assigned = df_assigned.drop(['DHT Time', 'CHT Time'], axis = 1)

# Data Updation after maintenance
df_assigned.Usage_Start= pd.to_datetime(df_assigned.Usage_Start, format = '%d/%m/%Y %H:%M:%S')
df_assigned.Usage_End= pd.to_datetime(df_assigned.Usage_End, format = '%d/%m/%Y %H:%M:%S')

df_assigned = df_assigned.sort_values(['Resource', 'Usage_End']).reset_index(drop= True)
df_assigned['Utilized'] = df_assigned['Usage_End'] - df_assigned['Usage_Start']
df_assigned['Utilized'] = df_assigned['Utilized']/(np.timedelta64(1, 's')*3600)

temp_df = pd.DataFrame()
for res in df_assigned.Resource.unique():
    data_temp = df_assigned[df_assigned['Resource'] == res]
    data_temp = data_temp.sort_values('Usage_Start')
    # Shift the data 1 level up to get perform the substraction operation
    data_temp['Next Usage'] = data_temp['Usage_Start'].shift(-1)
    data_temp['Unutilized_gap'] = data_temp['Next Usage'] - data_temp['Usage_End']
    temp_df = pd.concat([temp_df, data_temp])
df_assigned = temp_df.reset_index(drop = True)
del(temp_df)
print(check_overlaps(df_assigned))
df_assigned['Unutilized_gap'] = df_assigned['Unutilized_gap']/(np.timedelta64(1, 's')*3600)
df_assigned['idle_time'] = df_assigned['Unutilized_gap']-data1['Washing Time']

df_assigned['Max DHT Time'] = df_assigned['Usage_End'] + pd.to_timedelta(df_assigned['Max DHT']*3600, unit='s')
df_assigned['Last Possible Clean'] = df_assigned['Next Usage'] - pd.to_timedelta(df_assigned['Washing Time']*3600, unit='s')
df_assigned['Last Cleaning_DHT'] = df_assigned['Max DHT Time'] - pd.to_timedelta(df_assigned['Washing Time']*3600, unit='s')
cond = (df_assigned['Last Possible Clean']> df_assigned['Last Cleaning_DHT']) | (df_assigned['Last Possible Clean'].isnull())
df_assigned['Last Possible Clean'] = np.where(cond, df_assigned['Last Cleaning_DHT'], df_assigned['Last Possible Clean'])

df_assigned['Start Cleaning_CHT'] = df_assigned['Next Usage'] - pd.to_timedelta(df_assigned['Max CHT']*3600, unit='s')
df_assigned['Start Cleaning_CHT'] = df_assigned['Start Cleaning_CHT'] - pd.to_timedelta(df_assigned['Washing Time']*3600, unit='s')

res_adjust = ['P1', 'P2', 'P3', 'P4', 'SA', 'SB', 'SC', 'SD', 'AS26 TD',
              'I/N 97', 'I/N 2224', 'I/N 2667','I/N 2315', 'I/N 1619','I/N 2668','I/N 2316','I/N 1554', 
              'Dome 121', 'Dome 89', 'Dome 2215']

cond1 = (df_assigned['Start Cleaning_CHT']> df_assigned['clean_start']) & df_assigned['Resource'].isin(res_adjust) & (~df_assigned['Start Cleaning_CHT'].isnull())

df_assigned['clean_start'] = np.where(cond1, df_assigned['Start Cleaning_CHT'], df_assigned['clean_start'])
# Assuming the cleaning time is exactly as the defined time
df_assigned['clean_end'] = df_assigned['clean_start'] + pd.to_timedelta(df_assigned['Washing Time']*3600, unit='s')
df_assigned['Last_pos_preclean'] = df_assigned['Usage_Start'] - pd.to_timedelta(df_assigned['Washing Time']*3600, unit='s') - pd.Timedelta(minutes = buffer/5)

### Data Transfermation to identify the CHT &  DHT constraints

In [ ]:
df_assigned['Clean_Start_Time'] = pd.to_datetime(df_assigned['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
df_assigned['Clean_End_Time'] = pd.to_datetime(df_assigned['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
df_assigned['CHT Adjusted'] = np.nan
df_assigned = calculate_constraint(df = df_assigned.copy())

df_assigned = calculate_constraint_preclean(df = df_assigned.copy())
df_assigned = pd.concat([df_assigned, df_rec_performed]).reset_index(drop = True)
data_wo_cht_adj = df_assigned.copy()
data_cht_adj = df_assigned[df_assigned['CHT_Adjust']==1].sort_values('Usage_End')
df_assigned.shape

In [ ]:
df_cip_gconst = df_cip_gconst.rename(columns = {'value': 'Constraint'})

data_cht_adj['Clean_Start_Time'] = np.nan; data_cht_adj['Clean_End_Time'] = np.nan
data_cht_adj['MC Group'] = np.nan; data_cht_adj['Constraint'] = np.nan


count = 0
for adj_index, adj_row in data_cht_adj.iterrows():
    
    adj_res = adj_row['Resource']; adj_wash_time = adj_row['Washing Time']
    print(adj_res)
    idle_adj_start = adj_row['Start Cleaning_CHT']; dle_adj_last = adj_row['Last Cleaning_DHT']
    idle_adj_end = idle_adj_start + pd.Timedelta(minutes = adj_wash_time*60)
    
    mc_avail_df = cleaning_availablity(df = data_wo_cht_adj.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
                                       max_clean = max_clean, clean_dur = adj_wash_time,
                                       buffer_mins = buffer/5) 
    # Transfermed Constraint cleaning dataset
    const_avail_df = cleaning_availablity(df = data_wo_cht_adj.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint',
                                          max_clean = max_clean, clean_dur = adj_wash_time,
                                          buffer_mins = 0)
    rows = 0
    for mins in range(0,1000): 
        # Move the slot(5 mins step)
        idle_adj_start = adj_row['Start Cleaning_CHT'] + pd.Timedelta(minutes=(mins*1))
        idle_adj_last = adj_row['Last Cleaning_DHT']
        idle_adj_end = idle_adj_start + pd.Timedelta(minutes = adj_wash_time*60)
        # No CIP can be assigned if the time has reached to last cleaning time
        if idle_adj_start > idle_adj_last:
            idle_adj_start_new = adj_row['Start Cleaning_CHT']
            idle_adj_end_new = idle_adj_start_new + pd.Timedelta(minutes = adj_wash_time*60)
            print('<<<<<<<<<<<<<<<<NO SLOT>>>>>>>>>>>>>>>')
#             rows = 1
            break
        else:
            rows, data_cht_adj = clean_allocate(df_mc = mc_avail_df, df_cons = const_avail_df, res = adj_res,
                                                df_update = data_wo_cht_adj.copy(), index = adj_index, loop_df = data_cht_adj, 
                                                main_flag = 0, mc_const_df = df_cip_gconst.copy(), clean_start = idle_adj_start, 
                                                clean_end = idle_adj_end, m_start = None, m_end = None, th_time = idle_adj_last, 
                                                flag_var = "CHT Adjusted", buffer_time= buffer/5)
            if rows>0:
                data_wo_cht_adj = data_wo_cht_adj.drop(adj_index, axis = 0)
                data_wo_cht_adj = pd.concat([data_wo_cht_adj, data_cht_adj.loc[[adj_index],:]])
                
                print('success')
                break


In [ ]:
df_rec_performed = data_wo_cht_adj[data_wo_cht_adj['Cleaning Type']=='Reclean'].reset_index(drop = True)
data_wo_cht_adj = data_wo_cht_adj[data_wo_cht_adj['Cleaning Type']!='Reclean'].reset_index(drop = True)

# data_wo_cht_adj['Max DHT'] = data_wo_cht_adj['Max DHT']- (10/60)
data_wo_cht_adj['Clean_Start_Time'] = pd.to_datetime(data_wo_cht_adj['Clean_Start_Time'], format = '%d/%m/%Y %H:%M:%S')
data_wo_cht_adj['Clean_End_Time'] = pd.to_datetime(data_wo_cht_adj['Clean_End_Time'], format = '%d/%m/%Y %H:%M:%S')
data_wo_cht_adj.shape

### Pre Cleaning Handle

In [ ]:
data_assigned1 = calculate_constraint(df = data_wo_cht_adj.copy())
data_assigned2 = calculate_constraint_preclean(df = data_assigned1)

data_assigned_shift = Utilization_Shifter(df = data_assigned2.copy())
data_assigned1 = calculate_constraint(df = data_assigned_shift.copy())
data_assigned2 = calculate_constraint_preclean(df = data_assigned1).reset_index(drop = True)
data_assigned2.shape

In [ ]:
reclean_df = data_assigned2[(data_assigned2['CHT Violation Flag Pre Clean']==1)]
reclean_df = reclean_df.sort_values('clean_start')

In [ ]:
############################ change 06/12/2021
if len(df_qcip)==0:
    print("I am in If")
    min_clean = data.Usage_Start.min()
    min_cht_clean = data[data['Usage_Start']==min_clean].iloc[0]['Max CHT']
    min_clean = min_clean - pd.Timedelta(minutes=min_cht_clean*60)
else:
    min_clean = min(data.Clean_Start_Time.min(),data.Usage_Start.min())
    try:
        min_cht_clean = data[data['Usage_Start']==min_clean].iloc[0]['Max CHT']
        min_clean = min_clean - pd.Timedelta(minutes=min_cht_clean*60)
    except:
        print("I am in except")
        min_cht_clean = data[data['Usage_Start']==data.Usage_Start.min()].iloc[0]['Max CHT']
        min_usage_clean = data.Usage_Start.min() - pd.Timedelta(minutes=min_cht_clean*60)
        min_clean = min(data.Clean_Start_Time.min(),min_usage_clean)
############################ change end 06/12/2021       

In [ ]:
reclean_df['DHT Violation Flag'] = 0
reclean_df['CHT Violation Flag'] = 0
reclean_df['run_flag'] = 0
reclean_df['Initial Assignment'] = 0
reclean_df['Push Assignment'] = 0
reclean_df['CHT Adjusted'] = 0
data_assigned2['CHT Violation Flag'] = 0
reclean_df['Pre Clean Start'] = np.where(reclean_df['Pre Clean Start']<=reclean_df['Last Clean End'], 
                                        reclean_df['Last Clean End'], reclean_df['Pre Clean Start'])
reclean_df['Pre Clean Start'] = pd.to_datetime(reclean_df['Pre Clean Start'], 
                                                format = '%d/%m/%Y %H:%M:%S').dt.round('S').astype('datetime64[s]')

reclean_df['preclean'] = 1
reclean_df['Clean_Start_Time'] = np.nan; reclean_df['Clean_End_Time'] = np.nan
reclean_df['MC Group'] = np.nan; reclean_df['Constraint'] = np.nan

In [ ]:
count = 0
for i, r in reclean_df.iterrows():
    reclean_record = df_rec_performed[(df_rec_performed['Resource'] == r['Resource']) & (df_rec_performed['LOT'] == r['LOT']) & 
                                      (df_rec_performed['Type']== r['Type'])].reset_index(drop = True)
    if reclean_record.shape[0]>0:
        count = count+1
        reclean_record = reclean_record.iloc[0]
        reclean_df.loc[i, 'Clean_Start_Time'] = reclean_record['Clean_Start_Time']
        reclean_df.loc[i, 'Clean_End_Time'] = reclean_record['Clean_End_Time']
        reclean_df.loc[i, 'MC Group'] = reclean_record['MC Group']
        reclean_df.loc[i, 'Constraint'] = reclean_record['Constraint']
        reclean_df.loc[i, 'run_flag'] = 0
        reclean_df.loc[i, 'CHT Violation Flag Pre Clean'] = 0
        reclean_df.loc[i, 'Parallel Clean Flag'] = reclean_record['Parallel Clean Flag']
        data_assigned2.loc[i,'CHT Violation Flag'] = 0

reclean_filled = reclean_df[~reclean_df['MC Group'].isnull()]
reclean_filled['Cleaning Type'] = 'Recleaning'
data_assigned2['Cleaning Type'] = np.where(data_assigned2['Cleaning Type'].isnull(), 
                                           'Cleaning', 'Freezed')
data_assigned2 = pd.concat([data_assigned2, reclean_filled])
data_assigned2.shape

In [ ]:
data_assigned2 = calculate_constraint(df = data_assigned2.copy())
data_assigned2 = calculate_constraint_preclean(df = data_assigned2).reset_index(drop = True)
reclean_df = data_assigned2[(data_assigned2['CHT Violation Flag Pre Clean']==1) & (data_assigned2['preclean'].isnull())]
reclean_df = reclean_df.sort_values('clean_start')

In [ ]:
for i, r in reclean_df.iterrows():
    if data_freeze[(r['LOT']==data_freeze['LOT']) &  (r['Resource']==data_freeze['Resource']) & 
                  (r['Type'] == data_freeze['Type'])].shape[0]>0:
        reclean_df = reclean_df.drop(i)

In [ ]:
x_df_remaining = df_cip_gconst.rename(columns = {'value': 'Constraint'})
reclean_df['preclean'] = 1
reclean_df['Clean_Start_Time'] = np.nan; reclean_df['Clean_End_Time'] = np.nan
reclean_df['MC Group'] = np.nan; reclean_df['Constraint'] = np.nan
count = 0
for reclean_index, reclean_row in reclean_df.iterrows():
    count = count+1
    reclean_wash_time = reclean_row['Washing Time']
    reclean_res = reclean_row['Resource']
    print(reclean_res)
    # Transfermed CIP cleaning dataset
    mc_avail_df = cleaning_availablity(df = data_assigned2.copy(), sort_by_col = 'Clean_Start_Time', 
                                        sub_col =  'Clean_End_Time', for_col = 'MC Group', max_clean = max_clean, 
                                        clean_dur = reclean_wash_time, df_avail = df_cip_avail.copy(), 
                                       buffer_mins = buffer)
    print(count, reclean_index, mc_avail_df[mc_avail_df['Available_Time']<0].shape[0])

    # Transfermed Constraint cleaning dataset
    const_avail_df = cleaning_availablity(df = data_assigned2.copy(), sort_by_col = 'Clean_Start_Time', 
                                        sub_col =  'Clean_End_Time', for_col = 'Constraint', max_clean = max_clean, 
                                        clean_dur = reclean_wash_time, df_avail = df_const_avail.copy(),
                                          buffer_mins = 0)
    rows = 0
    if 'Flex' in reclean_res:
        pc_option = df_par_clean[df_par_clean['Process resource']==reclean_res]['PC Option'].tolist()
        df_par_pos = data_assigned2[(data_assigned2['Clean_Start_Time']>= reclean_row['Pre Clean Start']) &
                                    (data_assigned2['Resource'].isin(pc_option)) & 
                                    (data_assigned2['Parallel Clean Flag'].isnull()) &
                                    (data_assigned2['Last Possible Clean']<= reclean_row['Last_pos_preclean'])]
        if df_par_pos.shape[0]>0:
            par_count = par_count + 1
            index_list = df_par_pos.index
            data_assigned2.loc[index_list[0], 'Parallel Clean Flag'] = par_count
            reclean_df.loc[reclean_index, 'Clean_Start_Time'] = data_assigned2['Clean_Start_Time'][index_list[0]]
            reclean_df.loc[reclean_index, 'Clean_End_Time'] = data_assigned2['Clean_End_Time'][index_list[0]]
            reclean_df.loc[reclean_index, 'MC Group'] = data_assigned2['MC Group'][index_list[0]]
            reclean_df.loc[reclean_index, 'Constraint'] = data_assigned2['Constraint'][index_list[0]]
            reclean_df.loc[reclean_index, 'Parallel Clean Flag'] = data_assigned2['Parallel Clean Flag'][index_list[0]]
            reclean_df.loc[reclean_index, 'Reclean Status'] = 0
            rows = 1;  cht_violation_flag = 0
            print('Tata, Bye Bye, Gaya...')
        
    for mins in range(0,2500):
        if rows == 1:
            break
        idle_reclean_start = reclean_row['Pre Clean Start'] + pd.Timedelta(minutes=(mins*1))
        idle_reclean_end = idle_reclean_start + pd.Timedelta(minutes = reclean_wash_time*60)
        idle_reclean_last = reclean_row['Last_pos_preclean']
        # No CIP can be assigned if the time has reached to last cleaning time
        if idle_reclean_start > idle_reclean_last:
            idle_reclean_start_new = reclean_row['Last_pos_preclean']
            idle_reclean_end_new = idle_reclean_start_new + pd.Timedelta(minutes = reclean_wash_time*60)
            cht_violation_flag = 1
            print(reclean_res, '<<<<<<<<<<<<<<<< NO SLOT >>>>>>>>>>>>>>>')
            rows = 1
            break
        else:
            rows, reclean_df = clean_allocate(df_mc = mc_avail_df, df_cons = const_avail_df, res = reclean_res, 
                                              df_update = data_assigned2.copy(), index = reclean_index, 
                                              loop_df = reclean_df, main_flag = 0, mc_const_df = x_df_remaining.copy(),
                                              clean_start = idle_reclean_start, clean_end = idle_reclean_end, 
                                              m_start = None, m_end = None, th_time = idle_reclean_last, 
                                              reclean = 0, flag_var = 'Reclean Status', buffer_time= buffer/5)
        
        if rows>0:
            cht_violation_flag = 0
            print(mins)
            break
    if rows>0:
        reclean_df.loc[reclean_index,'run_flag'] = 0
        data_assigned2.loc[reclean_index,'CHT Violation Flag'] = cht_violation_flag
        reclean_df.loc[reclean_index, 'CHT Violation Flag Pre Clean'] = 0
        data_assigned2 = pd.concat([data_assigned2, reclean_df.loc[[reclean_index],:]])
    else:
        break

In [ ]:
df_cipc = cip_avail(df = data_assigned2.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
              sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

df_constc = cip_avail(df = data_assigned2.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
              sub_col = 'Clean_End_Time', max_clean = max_clean).drop('idle_till_time', axis = 1)

const_overlap = df_constc[df_constc['Available_Time']<0].shape[0]
dht_violation = data_assigned2[data_assigned2['Last Possible Clean']< data_assigned2['Clean_Start_Time']].shape[0]
mc_overlap = df_cipc[df_cipc['Available_Time']<0].shape[0]
(const_overlap + dht_violation + mc_overlap)

##### Plotting post handing the mandatory pre cleaning scenario

In [ ]:
data_assigned2[data_assigned2['Last Possible Clean']<data_assigned2['Clean_Start_Time']]

In [ ]:
data_assigned2['MC Group'] = data_assigned2['MC Group'].apply(lambda x: str(x).replace(' - Group', ''))
data_assigned2['MC Group'] = data_assigned2['MC Group'].replace('nan', np.nan)
replace_di = {'AS26 TD': 'AS26 TD Line - L2445', 'AS26 VII': 'AS26 VII Line - L2464'}
data_assigned2.Resource = data_assigned2.Resource.replace(replace_di) 


data_assigned2 = data_assigned2.rename(columns = {'Resource Exported': 'Resource Alt', 'Resource': 'Resource', 
                                                  'Type': 'Product Type'})

data_assigned2.loc[data_assigned2["Resource"].str.startswith('Flexible'), 'Resource Alt'] = 'Flexible'


# data_assigned2['Cleaning Type'] = 'Post Utilization Cleaning'
# # data_assigned2['Cleaning Type'] = np.where(~(data_assigned2['Parallel Clean Flag'].isnull()),
# #                                            'Parallel Cleaning', data_assigned2['Cleaning Type'])
# data_assigned2['Cleaning Type'] = np.where(data_assigned2['preclean']==1,
#                                            'Re-Cleaning', data_assigned2['Cleaning Type'])  
data_assigned_temp = data_assigned2.groupby('Resource', as_index= False)['Usage_Start'].min()
data_assigned_temp = data_assigned_temp.rename(columns = {'Usage_Start': 'Usage_Start_min'})
data_assigned2 = pd.merge(data_assigned2, data_assigned_temp, on = 'Resource', how = 'left')

 

In [ ]:
############# new section

In [ ]:
df = data_assigned2.copy()
x_df_remaining = df_cip_gconst.copy()
count = 0
for index, row in df_maintenance.iterrows():
    count = count + 1
    res = row['Resource']; start_time = row['Start_time']; end_time = row['End_time']; M_time = row['M_time']
    M_DHT = row['Max DHT']; M_code = row['Code']
    nearest_df = df[df['Resource']==res]
    nearest_df['Maintain_pre_CHT'] = start_time - pd.to_timedelta(nearest_df['Max CHT']*3600, unit='s')
    nearest_df = nearest_df[(nearest_df['Maintain_pre_CHT'] < nearest_df['Clean_End_Time']) & 
                            (nearest_df['Time_av_for_M'] > M_time) & (nearest_df['Cleaning Type'] != 'q-Pre Cleaning') ]
    washingtime = mapping_df[mapping_df['Resource'] == res].reset_index(drop = True)['Washing Time'][0]
    print(res, nearest_df.shape)
    ######################## change on 01/12/2021
    map_columns = ['MC Group', 'Constraint', 'Washing Time','Clean_Start_Time', 'Clean_End_Time','Parallel Clean Flag']
    ####################### change on 01/12/2021
    df_mc_group = df[map_columns]
    
    df_MC_available = cleaning_availablity(df = df_mc_group.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'MC Group', 
                                           sub_col = 'Clean_End_Time', max_clean = max_clean, clean_dur = washing_time, 
                                           buffer_mins = buffer/5)                                                              #why buffer/5?
    df_cons_available = cleaning_availablity(df = df_mc_group.copy(), sort_by_col = 'Clean_Start_Time', for_col = 'Constraint', 
                                             sub_col = 'Clean_End_Time', max_clean = max_clean, clean_dur = washing_time, 
                                             buffer_mins = 0)
    rows = 0
    if (nearest_df.shape[0]==0):
        print('Me Andar aya')
        blank_df = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns)
        ########################## new section added on 03/12/2021
        if df[df['Resource']==res].shape[0] == 0:
            print("I didn't had any utilization")
            start_time = start_time
            end_time = start_time + pd.Timedelta(minutes = M_time*60)
            blank_df = blank_df
            print(start_time, end_time)
            if len(df_cip_gconst[df_cip_gconst["Resource"]==res])==0:
                add_mapping= mapping_df[mapping_df["Resource"]==res]
                select_var = ['Resource']+allmcgroup
                add_mapping=add_mapping.loc[:, select_var]
                add_mapping = pd.melt(add_mapping, id_vars = add_mapping.columns[0], value_vars=add_mapping.columns[1:], var_name='MC Group')
                add_mapping = add_mapping[~(add_mapping['value'].isnull())]
                add_mapping = add_mapping[~(add_mapping['MC Group'].isnull())]
                add_mapping.columns = ["Resource","MC Group","Constraint"]
                df_cip_gconst = pd.concat([df_cip_gconst,add_mapping]).reset_index(drop = True)  
        ######################### new section ended on 03/12/2021
        for mins in range(0,1200):
#             print(mins)
            clean_assign_start = end_time + pd.Timedelta(minutes = mins*1)
            clean_assign_end = clean_assign_start + pd.Timedelta(minutes = washingtime*60)
            start_time = start_time + pd.Timedelta(minutes = mins*1)
            end_time = end_time + pd.Timedelta(minutes = mins*1)
            
            rows, blank_df = clean_allocate(df_mc = df_MC_available, df_cons = df_cons_available, res = res,
                                            df_update = df.copy(), index = 0, loop_df = blank_df, main_flag = 1, 
                                            mc_const_df = df_cip_gconst.copy(), clean_start = clean_assign_start, 
                                            clean_end = clean_assign_end, m_start = start_time, m_end = end_time, 
                                            th_time = max_clean, buffer_time= buffer/5)
               
            if rows>0:
                blank_df['MC Group'] = blank_df['MC Group'].str.replace(' - Group', '')
                print('Yes Mil gaya')
                clean_assign_start = blank_df['Clean_Start_Time'][0]
                end_time = clean_assign_start - pd.Timedelta(minutes=10)
                start_time = end_time -  pd.Timedelta(minutes=M_time*60)
                
                blank_df.loc[0,'Usage_Start'] = start_time
                blank_df.loc[0,'Usage_End'] = end_time
                blank_df.loc[0,'clean_flag'] = 1
                blank_df.loc[0,'run_flag'] = 0
                blank_df.loc[0,'Resource'] = res
                blank_df.loc[0,'LOT'] = M_code
                if df[df['Resource']==res].shape[0] == 0:
                    print("aya"+ str(df[df['Resource']==res].shape[0]))
                    blank_df.loc[0,'Last Possible Clean'] = max_clean
                    blank_df.loc[0,'DHT Violation Flag'] = 0
                    blank_df.loc[0,'CHT Violation Flag'] = 0
                df = pd.concat([df, blank_df.loc[[0],:]])
                break

In [ ]:
data_assigned2['Cleaning Type'] = np.where(data_assigned2['preclean']==1,
                                           'Recleaning', data_assigned2['Cleaning Type']) 
data_assigned2['Cleaning Type'] = np.where((data_assigned2['preclean']==1) & 
                                           (data_assigned2['Usage_Start']==data_assigned2['Usage_Start_min']),
                                           'Pre-Cleaning', data_assigned2['Cleaning Type'])  
data_assigned2['Cleaning Type'] = np.where((data_assigned2['Maint_Flag']==1),
                                           'Maintenance', data_assigned2['Cleaning Type']) 

In [ ]:
########### new section

In [ ]:
del_col = ['clean_start', 'clean_end', 'Last Cleaning_DHT', 'Start Cleaning_CHT',
           'Last_pos_preclean', 'Day Flag','Last_M_endtime', 'Time_av_for_M', 'CHT_Adjust', 
           'Last Clean End', 'Pre Clean Start']
data_assigned_pipe3 = data_assigned2.drop(del_col, axis = 1)
# data_assigned_pipe3['Washing Time'] = data_assigned_pipe3['Washing Time']*60
data_assigned_pipe3['Washing Time'] = data_assigned_pipe3['Clean_End_Time'] - data_assigned_pipe3['Clean_Start_Time']
data_assigned_pipe3['Washing Time'] = data_assigned_pipe3['Washing Time']/(np.timedelta64(1, 's')*60)

data_assigned_pipe3 = data_assigned_pipe3.rename(columns = {'idle_time' : 'Idle Time Ex Cleaning',
                                                            'Max DHT Time': 'End DHT Time',
                                                            'Washing Time': 'Cleaning Duration'})

data_assigned_pipe3['End CHT Time'] = data_assigned_pipe3['Clean_End_Time'] + pd.to_timedelta(data_assigned_pipe3['Max CHT']*3600, unit='s')

round_col =  ['Utilized', 'Unutilized_gap', 'Idle Time Ex Cleaning', 'Actual DHT', 'Actual CHT', 'Last CHT']
for col in round_col:
    data_assigned_pipe3[col] = data_assigned_pipe3[col].round(2) 
    



In [ ]:
order_col = ['LOT', 'Resource', 'Resource Alt', 'Product Type', 'Usage_Start', 'Usage_End', 'Utilized', 'Next Usage', 
             'Unutilized_gap', 'Cleaning Duration', 'Idle Time Ex Cleaning', 'Max DHT', 'Max CHT', 'End DHT Time', 
             'End CHT Time',  'Clean_Start_Time', 'Clean_End_Time', 'MC Group', 'Constraint', 'run_flag', 'clean_flag', 
             'Maint_Flag',  'Initial Assignment','Push Assignment','Actual DHT', 'Actual CHT', 'DHT Violation Flag', 
             'Last CHT', 'CHT Adjusted', 'Utilization Shifted', 'CHT Violation(Intermediate Gap)', 
             'CHT Violation Flag Pre Clean', 'CHT Violation Flag', 'preclean', 'Parallel Clean Flag',
             'Cleaning Type']

sort_col = ['Resource', 'Usage_Start', 'Clean_Start_Time']
data_assigned_pipe4 = data_assigned_pipe3[order_col].sort_values(sort_col).reset_index(drop = True)
flag_col = ['preclean', 'Maint_Flag', 'Initial Assignment','Push Assignment', 'CHT Adjusted']
data_assigned_pipe4[flag_col] = data_assigned_pipe4[flag_col].fillna(0)

In [ ]:
pre_final_df = data_assigned_pipe4[['Resource', 'LOT', 'Clean_Start_Time', 'Cleaning Duration', 'MC Group',
                                    'Usage_Start', 'Usage_End', 'Clean_End_Time', 'End DHT Time', 'End CHT Time', 
                                    'Constraint', 'Cleaning Type']]
df_maintenance = df_maintenance.rename(columns = {'Code': 'LOT'})
pre_final_df = pd.merge(pre_final_df, df_maintenance[['LOT', 'Resource', 'Start_time', 'End_time']], 
                        on = ['LOT', 'Resource'], how = 'left')

pre_final_df = pre_final_df.rename(columns = {'Start_time':'Old_Maintenance_Start_Time', 
                                              'End_time':'Old_Maintenance_End_Time'})

In [ ]:
columns_exp_maint = ['Resource', 'LOT', 'Usage_Start', 'Usage_End', 'Old_Maintenance_Start_Time', 'Old_Maintenance_End_Time']
df_maint = pre_final_df[pre_final_df['Cleaning Type']=='Maintenance'][columns_exp_maint]
pre_final_df = pre_final_df.drop(columns_exp_maint[-2:], axis = 1)

In [ ]:
import datetime 
time_now = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M")

In [ ]:
output_path  = r'C:\Users\AtulPoddar\OneDrive - TheMathCompany Private Limited\Documents\Takeda\testing\lot_code_test3'

export_file_name = output_path +'\\'+today+'_Takeda_Priority Production planning_Outcomev13.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(export_file_name, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
data_assigned_pipe4.to_excel(writer, sheet_name='Optimized Schedule Output (R)',index = False)
pre_final_df.to_excel(writer, sheet_name='Input ->Infor(OFD)', index = False)
df_maint.to_excel(writer, sheet_name='Maintenance ->Infor(OFD)', index = False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
schedule = pd.read_csv(r"C:\Users\AtulPoddar\OneDrive - TheMathCompany Private Limited\Documents\Takeda\testing\lot_code_test3\Schedule.csv")

In [ ]:
#################### generate gant chart#######################################################

In [ ]:
import pandas as pd
from pathlib import Path

from io import StringIO
import json

from pathlib import Path

from io import StringIO
from datetime import datetime, timedelta
import os
import pandas as pd
import json
from dateutil import parser
import plotly
import plotly.figure_factory as ff


In [ ]:
schedule=schedule.reset_index(drop=True)
def filter_data(selected_filter, df):
    cols_in_df = list(df.columns)
    # Filtering data
    if len(selected_filters) > 0:
        for key in selected_filters.keys():
            if type(selected_filters[key])==dict and selected_filters[key].get('start_date',False) and selected_filters[key].get('end_date',False):
                selected_filters[key]['start_date']=pd.to_datetime(selected_filters[key]['start_date'],infer_datetime_format=True,utc=True)
                selected_filters[key]['end_date']=pd.to_datetime(selected_filters[key]['end_date'],infer_datetime_format=True,utc=True)
                print(selected_filters)
                df=df[(df['Start']>selected_filters[key]['start_date']) & (df['Finish']<selected_filters[key]['end_date'])]
            elif key in cols_in_df:
                if type(selected_filter[key]) == str or type(selected_filter[key]) == int:
                    if selected_filter[key] == 'All':
                        continue
                    else:
                        df = df[df[key] == selected_filter[key]]
                else:
                    if isinstance(selected_filter[key],list) and 'All' in selected_filter[key]:
                        continue
                    else:
                        df = df[df[key].isin(selected_filter[key])]
            else:
                continue
    return df
util_features=['Resource','Usage_Start','Usage_End','Cleaning Type','MC Group','Cleaning Duration']
clean_features=['Resource','Clean_Start_Time','Clean_End_Time','Cleaning Type','MC Group','Cleaning Duration']
util_schedule=schedule[util_features]
util_schedule=util_schedule.rename(columns={'Resource':'Task','Usage_Start':'Start','Usage_End':'Finish'})
options=['Pre-Cleaning','Recleaning']
util_schedule['Resource']=np.where((util_schedule['MC Group'].isnull()) & (util_schedule['Cleaning Type'].isin(options)),'Utilization Post CHT violation','Utilization')
util_schedule['Resource']=np.where((util_schedule['Cleaning Type']=='Cleaning') & (util_schedule['Cleaning Duration'].isnull()) & (util_schedule['Resource']=='Utilization'),'Cleaning Post DHT violation',util_schedule['Resource'])
clean_schedule=schedule[clean_features]
clean_schedule=clean_schedule.rename(columns={'Resource':'Task','Clean_Start_Time':'Start','Clean_End_Time':'Finish','Cleaning Type':'Resource'})
final_schedule=pd.concat([util_schedule,clean_schedule])
final_schedule['Start']= pd.to_datetime(final_schedule['Start'],infer_datetime_format=True,utc=True)
final_schedule['Finish']= pd.to_datetime(final_schedule['Finish'],infer_datetime_format=True,utc=True)
final_schedule=final_schedule.rename(columns={'MC Group':'Description'})
final_schedule=filter_data(selected_filters,final_schedule)
final_schedule['Task'] = np.where((final_schedule['Task']=='P6 - Cutting&Loading'),'P6',final_schedule['Task'])
final_schedule['Task'] = np.where((final_schedule['Task']=='P5 - Cutting&Loading'),'P5',final_schedule['Task'])
final_schedule['Resource']=np.where(final_schedule['Resource']=='Freezed','Cleaning',final_schedule['Resource'])
sorter=['P5','P6','T78','T79','T80','T81','T82','P1','P2','P3','P4','SA','SB','SC','SD','Short Line - L717S','AS26 TD Line - L2445','Factor Line - L777','AS16 Line - L2103','AS26 VII Line - L2464','Long Transfer Line - L717L','4F Line - L2437','Dome 121','Dome 2215','Dome 89','615','616','617','I/N 1619','I/N 2668','I/N 97','I/N 2315','I/N 2667','I/N 2224','Flexibles - Cryo KIT A','Flexibles - Cryo KIT B',
'Flexibles - Cryo KIT Colorati 1','Flexibles - Cryo KIT Colorati 2','Flexibles - Mass Capture 1/2','Flexibles - Mass Capture 3/4','Flexibles - Factor IX/FEIBA','Flexibles - AS16','Flexibles - AS26 VII','I/N 2316','I/N 1554','Gauthier Filter','T50','T51','T52','T53','T54','T55','T58','T59',
'T60','T61','T62','T63','T64','T65','Flexibles - Kit 4F','Flexibles - Kit UF/DF', 'Flexibles - Kit Colonna','Flexibles - Kit Eluato','Flexibles - Kit Lavaggio','Flexibles - Kit Manifold','Flexibles - DDCPP']
final_schedule['Task'] = pd.Categorical(final_schedule['Task'], categories = sorter)
final_schedule=final_schedule.sort_values('Task').reset_index(drop= True)
colors = {'Pre-Cleaning': 'rgb(153, 0, 255)',
          'Utilization': 'rgb(0, 255, 100)',
          'Cleaning':(1, 0.9, 0.16),
          'Recleaning':'rgb(0, 0, 220)',
          'Cleaning Post DHT violation':'rgb(220, 0, 0)',
           'Utilization Post CHT violation':'rgb(255, 140, 0)',
           'Maintenance':'rgb(100, 50, 100)'}
fig = ff.create_gantt(final_schedule, colors=colors, index_col='Resource', show_colorbar=True,
                      group_tasks=True)
fig.update_xaxes(
    tickformat="%d-%b-%Y %H:%M")
fig.layout.xaxis.rangeselector = None
dynamic_outputs = plotly.io.to_json(fig)
fig.show()